# Justice League Stellar Merger History

## Charlotte Christensen, June 11 2025

This jupyter notebook runs Anna Wright's code (/home/christenc/Code/python/AnnaWrite_startrace/RomZoomSHAnalysisScripts) to identify the stars in those halos.

From Anna's Email (June 4, 2025)

I managed to get through the pipeline and remember what everything did well enough to comment it today, but didn't have time to test it on a new halo. However, if you'd like to try it in the next 24 hours or so, I've attached a zip file with steps 1-7 of my pipeline (there are 8 files, but TrackDownStars_rz.ipynb and FixHostIDs_rz.py are really two halves of a single step). Step 0 is creating a tangos db for the simulation you'll be working with and my pipeline uses that and the simulation itself to create an hdf5 file with relevant data for each star particle. The most important bits are a unique host ID for each star particle so that stars that formed in the same halo can be grouped together, even if that halo doesn't exist at z=0, and the orbital circularity of each star particle, which I use to identify members of the stellar halo.
I will be testing it tomorrow on one of the newer Romulus Zooms, so there's a good chance I'll be sending you an updated version very soon with any bug fixes :) 
I apologize for how many pieces the pipeline is in - it really isn't all that complicated, but it was adapted from what I did for the FOGGIE sims and I split the steps of that pipeline up so that I could do as much as possible locally (rather than on Pleiades) and so that I could sanity check as often as possible. Please let me know if you have any issues, questions, or suggestions! I'd definitely be eager to hear what Juan does differently!

In [4]:
import os
import socket
hostname = socket.gethostname()
if 'emu' in hostname:
    # os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/ns1917/tangos_sims/rogue.4096g5HbwK1BH_bn/'
    os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/ns1917/tangos_sims/'
    # os.environ['TANGOS_DB_CONNECTION'] = '/home/ns1917/Databases/Marvel_BN_N10.db' 
    os.environ['TANGOS_DB_CONNECTION'] = '/home/ns1917/pynbody/Tangos/Marvel_BN_N10.db'
    os.chdir('/home/ns1917/pynbody/AnnaWright_startrace/')
else: # grinnell
    os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
    # os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/Data/Marvel_BN_N10.db'
    os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/pynbody/Tangos/Marvel_BN_N10.db'
    os.chdir('/home/selvani/MAP/pynbody/AnnaWright_startrace/')

import tangos
tangos.all_simulations()

[<Simulation("cptmarvel.4096g5HbwK1BH_bn")>,
 <Simulation("rogue.4096g5HbwK1BH_bn")>,
 <Simulation("storm.4096g5HbwK1BH_bn")>,
 <Simulation("elektra.4096g5HbwK1BH_bn")>]

In [3]:
!tangos serve

Starting server in PID 3503746.
2025-08-13 14:03:59,110 INFO  [waitress:449][MainThread] Serving on http://[::1]:6543
2025-08-13 14:03:59,110 INFO  [waitress:449][MainThread] Serving on http://127.0.0.1:6543
2025-08-13 14:04:07,913 : Tree build complete; total time 3.53s
2025-08-13 14:04:07,913 INFO  [tangos.log:72][waitress-1] Tree build complete; total time 3.53s
2025-08-13 14:04:07,913 :   Progenitor query took 1.95s
2025-08-13 14:04:07,913 INFO  [tangos.log:73][waitress-1]   Progenitor query took 1.95s
2025-08-13 14:04:07,913 :   Property query took 0.02s
2025-08-13 14:04:07,913 INFO  [tangos.log:74][waitress-1]   Property query took 0.02s
2025-08-13 14:04:07,913 :   Tree post-processing took 1.56s
2025-08-13 14:04:07,913 INFO  [tangos.log:75][waitress-1]   Tree post-processing took 1.56s
2025-08-13 14:10:38,854 : Tree build complete; total time 0.06s
2025-08-13 14:10:38,854 INFO  [tangos.log:72][waitress-0] Tree build complete; total time 0.06s
2025-08-13 14:10:38,854 :   Progenit

In [2]:
# tangos.get_simulation("cptmarvel.4096g5HbwK1BH_bn").timesteps

In [5]:
import pynbody as pb
import numpy as np
import pandas as pd
import glob
import h5py
import tangos
import tqdm.auto as tqdm

# For importing modules
import importlib.util
import sys
from pathlib import Path

# Import the module
# base_path = '/home/selvani/MAP/pynbody/'
if 'emu' in hostname:
    base_path = '/home/ns1917/pynbody/'
else:
    base_path = '/home/selvani/MAP/pynbody/'

for root, dirs, files in os.walk(base_path):
    if root not in sys.path:
        print("Adding to sys.path:", root)
        sys.path.append(root)

Adding to sys.path: /home/ns1917/pynbody/
Adding to sys.path: /home/ns1917/pynbody/.git
Adding to sys.path: /home/ns1917/pynbody/.git/branches
Adding to sys.path: /home/ns1917/pynbody/.git/info
Adding to sys.path: /home/ns1917/pynbody/.git/hooks
Adding to sys.path: /home/ns1917/pynbody/.git/refs
Adding to sys.path: /home/ns1917/pynbody/.git/refs/heads
Adding to sys.path: /home/ns1917/pynbody/.git/refs/tags
Adding to sys.path: /home/ns1917/pynbody/.git/refs/remotes
Adding to sys.path: /home/ns1917/pynbody/.git/refs/remotes/origin
Adding to sys.path: /home/ns1917/pynbody/.git/objects
Adding to sys.path: /home/ns1917/pynbody/.git/objects/pack
Adding to sys.path: /home/ns1917/pynbody/.git/objects/info
Adding to sys.path: /home/ns1917/pynbody/.git/objects/f2
Adding to sys.path: /home/ns1917/pynbody/.git/objects/fe
Adding to sys.path: /home/ns1917/pynbody/.git/objects/16
Adding to sys.path: /home/ns1917/pynbody/.git/objects/a7
Adding to sys.path: /home/ns1917/pynbody/.git/objects/59
Adding t

In [6]:
# Simulation name and path
if 'emu' in hostname:
    simpath = '/home/ns1917/tangos_sims/'
    outfile_dir = "/home/ns1917/pynbody/stellarhalo_trace_aw/"
else:
    simpath = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
    outfile_dir = "/home/selvani/MAP/pynbody/stellarhalo_trace_aw/"

# basename = 'cptmarvel.cosmo25cmb.4096g5HbwK1BH'
# basename = 'rogue.cosmo25cmb.4096g5HbwK1BH'
basename = 'storm.cosmo25cmb.4096g5HbwK1BH'
# ss_dir = 'cptmarvel.4096g5HbwK1BH_bn'#'snapshots_200crit_h329' # same as db_sim?
# ss_dir = 'rogue.4096g5HbwK1BH_bn'
ss_dir = 'storm.4096g5HbwK1BH_bn'
sim_base = simpath + ss_dir + '/'
ss_z0 = sim_base + basename + '.004096'

#### Tangos tests

In [29]:
all_timesteps = tangos.get_simulation(ss_dir).timesteps

In [30]:
timestep = all_timesteps[-1]

In [31]:
halos = timestep.halos.all()

In [32]:
halos_with_stars = [halo for halo in halos if halo.NStar > 0]
halos_with_stars

[<Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_1' | NDM=13623527 Nstar=6026197 Ngas=3275306>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_2' | NDM=5955357 Nstar=6256 Ngas=2380301>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_3' | NDM=3597007 Nstar=436496 Ngas=1240096>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_5' | NDM=2740129 Nstar=463591 Ngas=1284025>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_7' | NDM=2677180 Nstar=146577 Ngas=420505>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_8' | NDM=2183798 Nstar=322455 Ngas=303863>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_9' | NDM=1811360 Nstar=156759 Ngas=343049>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g5HbwK1BH.004096/halo_10' | NDM=1904403 Nstar=88437 Ngas=52508>,
 <Halo 'rogue.4096g5HbwK1BH_bn/rogue.cosmo25cmb.4096g

In [33]:
halo = halos_with_stars[0]

In [34]:
halo.all_properties

[<HaloProperty Mvir=7.04e+10 of <Halo 1 of ...>>,
 <HaloProperty Rvir=8.65e+01 of <Halo 1 of ...>>,
 <HaloProperty Xc=8.61e+03 of <Halo 1 of ...>>,
 <HaloProperty Yc=6.52e+03 of <Halo 1 of ...>>,
 <HaloProperty Zc=8.11e+03 of <Halo 1 of ...>>,
 <HaloProperty VXc=-5.29e+00 of <Halo 1 of ...>>,
 <HaloProperty VYc=-5.81e+01 of <Halo 1 of ...>>,
 <HaloProperty VZc=1.77e+01 of <Halo 1 of ...>>,
 <HaloProperty Vmax=8.40e+01 of <Halo 1 of ...>>,
 <HaloProperty fMhires=1.00e+00 of <Halo 1 of ...>>,
 <HaloProperty M_gas=2.94e+09 of <Halo 1 of ...>>,
 <HaloProperty M_star=1.20e+09 of <Halo 1 of ...>>,
 <HaloProperty n_gas=3275306 of <Halo 1 of ...>>,
 <HaloProperty n_star=6026197 of <Halo 1 of ...>>,
 <HaloProperty n_dm=13623527 of <Halo 1 of ...>>,
 <HaloProperty npart=22925030 of <Halo 1 of ...>>,
 <HaloProperty shrink_center (array) of <Halo 1 of ...>>,
 <HaloProperty max_radius=1.18e+02 of <Halo 1 of ...>>]

In [35]:
for halo in halos_with_stars:
    if halo['Mvir'] > 1e9:
        print(f"Halo ID: {halo.halo_number}, M_star: {halo['M_star']:.2e}, NStar: {halo.NStar}, Mvir: {halo['Mvir']:.2e}")

Halo ID: 1, M_star: 1.20e+09, NStar: 6026197, Mvir: 7.04e+10
Halo ID: 2, M_star: 1.22e+06, NStar: 6256, Mvir: 1.98e+12
Halo ID: 3, M_star: 8.47e+07, NStar: 436496, Mvir: 1.88e+10
Halo ID: 5, M_star: 9.44e+07, NStar: 463591, Mvir: 9.83e+10
Halo ID: 7, M_star: 2.85e+07, NStar: 146577, Mvir: 1.35e+10
Halo ID: 8, M_star: 6.22e+07, NStar: 322455, Mvir: 1.10e+10
Halo ID: 9, M_star: 3.08e+07, NStar: 156759, Mvir: 3.64e+10
Halo ID: 10, M_star: 1.70e+07, NStar: 88437, Mvir: 9.33e+09
Halo ID: 11, M_star: 5.68e+06, NStar: 28972, Mvir: 6.92e+09
Halo ID: 12, M_star: 1.02e+07, NStar: 52777, Mvir: 6.66e+09
Halo ID: 13, M_star: 9.83e+02, NStar: 4, Mvir: 1.43e+11
Halo ID: 14, M_star: 1.04e+07, NStar: 52223, Mvir: 2.05e+10
Halo ID: 16, M_star: 1.30e+06, NStar: 6760, Mvir: 3.14e+09
Halo ID: 17, M_star: 2.47e+05, NStar: 1271, Mvir: 2.92e+09
Halo ID: 18, M_star: 1.60e+05, NStar: 822, Mvir: 2.76e+09
Halo ID: 23, M_star: 1.85e+06, NStar: 9116, Mvir: 1.15e+10
Halo ID: 26, M_star: 3.59e+05, NStar: 1784, Mvir: 

In [36]:
halos_with_stars = halos_with_stars[:8]

In [37]:
'''import tangos.examples.mergers as mergers

for halo in halos_with_stars:
    print("\nMain Progenitor z=0 Halo ID: %s" % (halo.halo_number))
    redshift, ratio, progenitor_halos = mergers.get_mergers_of_major_progenitor(halo)
    # each item of progenitor_halos is a pair; the first is the major progenitor, the second is the thing merging into it
    merging_structures = [x[1] for x in progenitor_halos if x[1].NStar > 0]
    print("There are %d mergers into the major progenitor branch.\n" % len(merging_structures))

    for i, merging_halo in enumerate(merging_structures):
        snapshot = merging_halo.timestep.extension[-6:]
        print("Merging Halo %d/%d at Snapshot %s" % (i+1, len(merging_structures), snapshot))

        merging_halo_num = merging_halo.halo_number
        progenitor_halo_num = progenitor_halos[i][0].halo_number
        # print(merging_halo.keys())
        # print("  Mass: %f" % merging_halo['Mvir'])
        print("  Merging Halo ID: %s, NStars: %d, NGas: %d" % (merging_halo_num, merging_halo.NStar, merging_halo.NGas))
        print("  Main Progenitor Halo ID: %s, NStars: %d, NGas: %d" % (progenitor_halo_num, progenitor_halos[i][0].NStar, progenitor_halos[i][0].NGas))
        # print("  Redshift: %f" % redshift[i])
        # print("  Ratio: %f" % ratio[i])
        print()'''

'import tangos.examples.mergers as mergers\n\nfor halo in halos_with_stars:\n    print("\nMain Progenitor z=0 Halo ID: %s" % (halo.halo_number))\n    redshift, ratio, progenitor_halos = mergers.get_mergers_of_major_progenitor(halo)\n    # each item of progenitor_halos is a pair; the first is the major progenitor, the second is the thing merging into it\n    merging_structures = [x[1] for x in progenitor_halos if x[1].NStar > 0]\n    print("There are %d mergers into the major progenitor branch.\n" % len(merging_structures))\n\n    for i, merging_halo in enumerate(merging_structures):\n        snapshot = merging_halo.timestep.extension[-6:]\n        print("Merging Halo %d/%d at Snapshot %s" % (i+1, len(merging_structures), snapshot))\n\n        merging_halo_num = merging_halo.halo_number\n        progenitor_halo_num = progenitor_halos[i][0].halo_number\n        # print(merging_halo.keys())\n        # print("  Mass: %f" % merging_halo[\'Mvir\'])\n        print("  Merging Halo ID: %s, NSta

In [41]:
s = pb.load(sim_base + basename + '.004032')
# h = s.halos(halo_numbers='v1')

unique_halos, counts = np.unique(s.s['amiga.grp'][s.s['tform'] > 0], return_counts=True)
iord_counts = pd.Series(counts, index=unique_halos)
print(iord_counts)

-1              5
 1        5894979
 3         431811
 5         444634
 7         145261
 8         320282
 9         155147
 10         88205
 11         28558
 12         52540
 13             2
 14         51781
 16          6722
 17          1258
 18           811
 24          8811
 26          1730
 30          3942
 31          5777
 32            28
 34          1633
 35           280
 36          6260
 37          6325
 60            27
 76             2
 123           51
 2997        1216
 3625         967
 7693         405
 10463        283
 12485        230
 25901         92
 33642         64
dtype: int64


In [14]:
s = pb.load(sim_base + basename + '.000480')
# h = s.halos(halo_numbers='v1')

unique_halos, counts = np.unique(s.s['amiga.grp_copy'][s.s['tform'] > 0], return_counts=True)
iord_counts = pd.Series(counts, index=unique_halos)
print(iord_counts)

-1       353194
 1       257970
 2        83526
 3        40725
 4         8061
 6         6073
 8          537
 9         1820
 10       16863
 11        3160
 13        4459
 14        1374
 15        3621
 16        1429
 17        3429
 20        1311
 21        5798
 23        1049
 24         358
 25          61
 26        1531
 27        2216
 28          42
 32        1285
 36         779
 38         326
 64          27
 88           2
 93          51
 113         28
 147        106
 157          5
 1895       912
dtype: int64


In [33]:
s = pb.load(sim_base + basename + '.000480')
h = s.halos(halo_numbers='v1')

unique_halos, counts = np.unique(s.s['amiga.grp'][s.s['tform'] > 0], return_counts=True)
iord_counts = pd.Series(counts, index=unique_halos)
print(iord_counts)

-1       353194
 1       257970
 2        83526
 3        40725
 4         8061
 6         6073
 8          537
 9         1820
 10       16863
 11        3160
 13        4459
 14        1374
 15        3621
 16        1429
 17        3429
 20        1311
 21        5798
 23        1049
 24         358
 25          61
 26        1531
 27        2216
 28          42
 32        1285
 36         779
 38         326
 64          27
 88           2
 93          51
 113         28
 147        106
 157          5
 1895       912
dtype: int64


In [14]:
sim_base + basename + '.000384'

'/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.000384'

In [32]:
s = pb.load(sim_base + basename + '.000384')
h = s.halos(halo_numbers='v1')

unique_halos, counts = np.unique(s.s['amiga.grp'][s.s['tform'] > 0], return_counts=True)
iord_counts = pd.Series(counts, index=unique_halos)
print(iord_counts)

1       233950
2       140971
3         5322
4        30720
5        21688
6         1555
8        16341
10        1995
11        2775
12        1010
13         542
14        1614
16        1341
18        1934
19         643
20        1137
21         832
23         728
24         595
25        2228
26        1248
30          14
33        1053
38         330
39          39
73          27
84           2
97          51
106         28
113        106
173       1649
1035        70
dtype: int64


In [24]:
!tangos serve

2025-07-30 10:54:24,889 INFO  [matplotlib.font_manager:1639][MainThread] generated new fontManager
Starting server in PID 2281410.
2025-07-30 10:54:25,406 INFO  [waitress:449][MainThread] Serving on http://[::1]:6543
2025-07-30 10:54:25,406 INFO  [waitress:449][MainThread] Serving on http://127.0.0.1:6543
2025-07-30 10:54:38,125 WARNI [waitress.queue:113][MainThread] Task queue depth is 1
2025-07-30 10:54:38,275 WARNI [waitress.queue:113][MainThread] Task queue depth is 1
2025-07-30 10:54:38,461 WARNI [waitress.queue:113][MainThread] Task queue depth is 1
2025-07-30 10:54:38,598 : Tree build complete; total time 0.13s
2025-07-30 10:54:38,598 INFO  [tangos.log:72][waitress-2] Tree build complete; total time 0.13s
2025-07-30 10:54:38,598 :   Progenitor query took 0.09s
2025-07-30 10:54:38,598 INFO  [tangos.log:73][waitress-2]   Progenitor query took 0.09s
2025-07-30 10:54:38,598 :   Property query took 0.01s
2025-07-30 10:54:38,598 INFO  [tangos.log:74][waitress-2]   Property query took 

In [20]:
h[1].derivable_keys()

['HII',
 'HeIII',
 'ne',
 'hetot',
 'hydrogen',
 'feh',
 'oxh',
 'ofe',
 'mgfe',
 'nefe',
 'sife',
 'c_s',
 'c_s_turb',
 'mjeans',
 'mjeans_turb',
 'ljeans',
 'ljeans_turb',
 'U_mag',
 'U_lum_den',
 'B_mag',
 'B_lum_den',
 'V_mag',
 'V_lum_den',
 'R_mag',
 'R_lum_den',
 'I_mag',
 'I_lum_den',
 'J_mag',
 'J_lum_den',
 'H_mag',
 'H_lum_den',
 'K_mag',
 'K_lum_den',
 'u_mag',
 'u_lum_den',
 'g_mag',
 'g_lum_den',
 'r_mag',
 'r_lum_den',
 'i_mag',
 'i_lum_den',
 'z_mag',
 'z_lum_den',
 'y_mag',
 'y_lum_den',
 'r',
 'rxy',
 'vr',
 'v2',
 'vt',
 'ke',
 'te',
 'j',
 'j2',
 'jz',
 'vrxy',
 'vcxy',
 'vphi',
 'vtheta',
 'v_mean',
 'v_disp',
 'v_curl',
 'vorticity',
 'v_div',
 'age',
 'theta',
 'alt',
 'az',
 'cs',
 'mu',
 'p',
 'u',
 'temp',
 'zeldovich_offset',
 'aform',
 'tform',
 'iord_argsort',
 'smooth',
 'rho']

### 1) GrabTF_rz.py

Step 1 of stellar halo pipeline

**What it does:**
- Loads the final snapshot (z=0) of the simulation
- Extracts formation time (`tform`) and particle IDs (`iord`) for all star particles that have `tform > 0` (i.e., actual stars, not wind particles)
- Converts formation times to Gyr units
- Creates a 2D array with particle IDs in the first row and formation times in the second row
- Saves this data as a `.npy` file named `<simulation_name>_tf.npy`
- Prints the total number of star particles found as a sanity check

**Input:** Simulation snapshot (specifically the final snapshot `.004096`)

**Output:** `<sim>_tf.npy` - NumPy file containing:
- Row 0: Star particle IDs (`iord`)  
- Row 1: Formation times in Gyr (`tform`)

**Purpose:** This creates the foundation dataset that subsequent steps will use to trace back each star particle to determine which halo it was forming in at its birth time.

* Usage:   `python GrabTF_rz.py <simpath> <output_directory>`
* Example: `python GrabTF_rz.py /path/to/sim/ /path/to/output/`
* Runtime:  ~1 min (cptmarvel), ~5 mins (rogue), ~3 mins (storm)

In [53]:
# # Import Anna's code, even if not along my path
# file_path = '/home/selvani/MAP/pynbody/AnnaWright_startrace/RomZoomSHAnalysisScripts/GrabTF_rz.py'
# module_name = 'GrabTF_rz'

# spec = importlib.util.spec_from_file_location(module_name, file_path)
# module = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(module)
import GrabTF_rz

In [54]:
GrabTF_rz.main(ss_z0, outfile_dir)

4045429 stars found!
Save to  /home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_tf.npy


### 2) LocAtCreation_pool_rz.py

<!-- Step 2 of stellar halo pipeline -->
<!-- Identifies the host of each star particle in \<sim\>_tf.npy at the  -->
<!-- time it was formed.  -->
<!-- **Note that what is stored is NOT the amiga.grp 
ID, but the index of that halo in the tangos database. The amiga.grp
ID can be backed out via tangos with sim[stepnum][halonum].finder_id.** 
(CC, I believe that I edited this so now the halo_) -->

<!-- Output: <sim>_stardata_<snapshot>.h5
        where <snapshot> is the first snapshot that a given process
        analyzed. There will be <nproc> of these files generated
        and processes will not necessarily analyze adjacent snapshots -->

<!-- * Usage:   python LocAtCreation_pool_rz.py <sim> optional:<nproc>
* Example: python LocAtCreation_pool_rz.py r634 2 -->

<!-- Includes an optional argument to specify number of processes to run
with; default is 4. Note that this will get reduced if you've specified
more processes than you have snapshots to process. -->

<!-- Note that this has the name of the snapshots directory hardcoded into FindHaloStars.py (L63)
-- Will need to be adjusted
The 

CC: When I did my edits, I moved code into FindHaloStars so that it can be imported for multiprocessing -->


Step 2 of stellar halo pipeline

**What it does:**
- Loads the `<sim>_tf.npy` file created in step 1 (containing star particle IDs and formation times)
- Queries the Tangos database to get all available simulation snapshots and their cosmic times
- Determines which snapshots contain newly formed stars by binning star formation times
- For each relevant snapshot, identifies which halo each star particle belonged to at the time it formed
- Extracts additional data for each star: formation position, formation time, and host halo ID
- Converts Amiga halo group IDs to Tangos database indices for consistency
- Handles unbound particles (those not in any halo) by assigning them host ID = -1

**Detailed Process:**
1. **Data Loading**: Loads the `_tf.npy` file containing star particle IDs and formation times
2. **Snapshot Analysis**: Gets all simulation timesteps from Tangos database and sorts by cosmic time
3. **Star Distribution**: Creates histogram of star formation times to identify which snapshots contain new stars
4. **Chunk Creation**: Divides snapshots among multiple processes for parallel processing
<!-- 5. **Multiprocessing Execution**: Each process handles a subset of snapshots independently -->

**FindHaloStars Function (called by each process):**
- **Time Matching**: For each snapshot, finds stars that formed between the previous snapshot and current one
- **Particle Matching**: Uses `iord` (particle IDs) to match stars from step 1 with their counterparts in historical snapshots
- **Host Identification**: Determines which halo (`amiga.grp`) each star was in when it formed
- **Database Indexing**: Converts halo IDs to Tangos database indices using a lookup dictionary
- **Position/Time Extraction**: Records formation positions, times, and snapshot locations
- **Data Writing**: Periodically saves data to HDF5 files to manage memory usage

<!-- **Key Technical Details:**
<!-- - Each process loads the same `_tf.npy` data independently to avoid sharing conflicts -->
<!-- - Uses `np.searchsorted()` for efficient particle ID matching between snapshots -->
<!-- - Creates `fid` dictionary to map Amiga group IDs to Tangos database indices
- Handles missing particles gracefully (assigns host ID = -1 for unbound stars -->

**Input:** 
- `<sim>_tf.npy` from step 1
- Simulation snapshots (all timesteps)
- Tangos database connection

**Output:** `<sim>_stardata_<snapshot>.h5` files (one per process) containing:
- `particle_IDs`: Star particle IDs (`iord`) of stars formed between snapshot and previous snapshot
- `particle_positions`: 3D positions at formation time (Mpc)
- `particle_creation_times`: Formation times (Gyr)
- `timestep_location`: Snapshot number where star was first found
- `particle_hosts`: Host halo index in Tangos database (-1 for unbound stars)

<!-- **Performance Features:**
- **Multiprocessing**: Uses all available CPU cores (up to 72 logical cores) for parallel processing
- **Load Balancing**: Randomly shuffles snapshot order to distribute work evenly
- **Memory Management**: Periodically writes data to disk to prevent memory overflow
- **Process Isolation**: Each process works independently to avoid conflicts -->

**Important Notes:**
<!-- - **Host IDs are Tangos database indices, NOT Amiga group IDs** -->
- Multiple output files are created (one per process) that will be combined in later steps. Just run with n_threads>num_snapshots to make one file per snapshot.
<!-- - Uses multiprocessing for significant speed improvement on multi-core systems -->
<!-- - Automatically handles load balancing by shuffling snapshot order -->
<!-- - Each process creates its own output file named by the first snapshot it processes -->

**Purpose:** This step creates the detailed formation history for each star particle, linking it to its birth halo and enabling stellar halo analysis. The multiprocessing approach significantly reduces computation time for large simulations.
* Usage: `python LocAtCreation_pool_rz.py <simpath> <db_sim_name> <output_dir> [n_processes]`
* Example: `python LocAtCreation_pool_rz.py /path/to/sim/ cptmarvel.4096g5HbwK1BH_bn /output/ 36`
* Runtime: ~1 hour sequential

In [5]:
import LocAtCreation_pool_rz

In [6]:
import psutil
n_cpus = psutil.cpu_count(logical=True) # use up to 36 quirm cores
LocAtCreation_pool_rz.main(simpath, basename, ss_dir, outfile_dir, 128, overwrite=True)

Stars from 42 steps left to deal with
Initializing  42
Shuffled chunks: [['storm.cosmo25cmb.4096g5HbwK1BH.003072']
 ['storm.cosmo25cmb.4096g5HbwK1BH.000960']
 ['storm.cosmo25cmb.4096g5HbwK1BH.000288']
 ['storm.cosmo25cmb.4096g5HbwK1BH.002688']
 ['storm.cosmo25cmb.4096g5HbwK1BH.001248']
 ['storm.cosmo25cmb.4096g5HbwK1BH.003168']
 ['storm.cosmo25cmb.4096g5HbwK1BH.002496']
 ['storm.cosmo25cmb.4096g5HbwK1BH.003456']
 ['storm.cosmo25cmb.4096g5HbwK1BH.002976']
 ['storm.cosmo25cmb.4096g5HbwK1BH.003744']
 ['storm.cosmo25cmb.4096g5HbwK1BH.003936']
 ['storm.cosmo25cmb.4096g5HbwK1BH.001824']
 ['storm.cosmo25cmb.4096g5HbwK1BH.001056']
 ['storm.cosmo25cmb.4096g5HbwK1BH.003648']
 ['storm.cosmo25cmb.4096g5HbwK1BH.002592']
 ['storm.cosmo25cmb.4096g5HbwK1BH.000864']
 ['storm.cosmo25cmb.4096g5HbwK1BH.001344']
 ['storm.cosmo25cmb.4096g5HbwK1BH.001920']
 ['storm.cosmo25cmb.4096g5HbwK1BH.000672']
 ['storm.cosmo25cmb.4096g5HbwK1BH.000768']
 ['storm.cosmo25cmb.4096g5HbwK1BH.001152']
 ['storm.cosmo25cmb.4096g

Processing:   0%|          | 0/42 [00:00<?, ?chunks/s]

Processing chunk 1/42: ['storm.cosmo25cmb.4096g5HbwK1BH.003072']
MyFirstStep:  003072
/home/ns1917/tangos_sims/
HaloStarsPath: /home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003072
86816 relevant stars in storm.cosmo25cmb.4096g5HbwK1BH.003072
Host Array: [8 8 1 ... 4 4 4]
Looking through 33154 halos
Amiga Host IDs [ 1  2  3  4  5  6  8  9 10 11 12 13 20 22 26 32 33 45 85]
FID Host IDs [ 1  2  3  4  5  6  8  9 10 11 12 13 20 22 26 32 33 45 85]
Process completed 1 snapshots, output: /home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_stardata_003072.h5
  Completed: 003072

Processing chunk 2/42: ['storm.cosmo25cmb.4096g5HbwK1BH.000960']
MyFirstStep:  000960
/home/ns1917/tangos_sims/
HaloStarsPath: /home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.000960
132460 relevant stars in storm.cosmo25cmb.4096g5HbwK1BH.000960
Host Array: [8 8 8 ... 4 4 4]
Looking through 33402 halos
Amiga Host IDs [   1    2    3    4

### 3) writeouthosts_rz.py

<!-- Step 3 of stellar halo pipeline                                                                                   
For each snapshot, writes out a list of halos that formed stars                                                   
between the last snapshot and this one and the number of stars formed;                                            
as in step 2, note that the IDs of these halos will be their index in                                             
the tangos database, not necessarily their amiga.grp ID. This is used                                             
to construct a unique ID for each star-forming halo in the next step.                                             
                                                                                                                   -->
<!-- Output: <sim>_halostarhosts.txt                                                                                   
                                                                                                                  
Usage:   python writeouthosts_rz.py <sim>                                                                         
Example: python writeouthosts_rz.py r634                                                                          
                                                                                                                   -->
<!-- Note that this is currently set up for MMs, but should be easily adapted                                          
by e.g., changing the paths or adding a path CL argument.     -->

Step 3 of stellar halo pipeline

**What it does:**
- Reads all the `<sim>_stardata_*.h5` files created in Step 2 
- For each simulation snapshot, identifies which halos formed new stars between that snapshot and the previous one
- Counts how many stars formed in each halo during each time interval
- Creates a comprehensive timeline of star formation activity across all halos
- Outputs a summary text file listing star-forming halos and their activity over snapshots

**Detailed Process:**
1. **File Discovery**: Locates all `*_stardata_*.h5` files from Step 2 using glob pattern matching
2. **Data Extraction**: For each HDF5 file, extracts:
   - `particle_hosts`: Host halo indices 
   - `timestep_location`: Snapshot numbers where each star particle first appears
3. **Temporal Binning**: Groups star particles by the snapshot where they formed
4. **Halo Counting**: For each snapshot, counts how many stars formed in each unique halo
5. **Output Formatting**: Creates chronologically ordered summary with format:
   ```
   <snapshot_number>    <halo_id_1>,<star_count_1>    <halo_id_2>,<star_count_2>    ...
   ```

<!-- **Key Technical Details:**
- **Vectorized Operations**: Uses `np.unique(return_counts=True)` for efficient halo counting instead of slow loops
- **Memory Efficient**: Processes one HDF5 file at a time to minimize memory usage
- **Chronological Ordering**: Sorts output by snapshot number for temporal analysis
- **Duplicate Handling**: Aggregates data from multiple stardata files that may have overlapping snapshots -->

**Input Files:**
- Multiple `<sim>_stardata_<snapshot>.h5` files from Step 2
- Each file contains star formation data for a subset of simulation snapshots

**Output File:** `<sim>_halostarhosts.txt`
- Text file with tab-separated values
- Each line represents one simulation snapshot
- Format: `<timestep>\t<halo_id>,<count>\t<halo_id>,<count>\t...`
- Example line: `3840    -1,234    1,156    3,89`
  - At snapshot 3840: halo -1 formed 234 unbound stars, halo 1 formed 156 stars, halo 3 formed 89 stars

<!-- **Data Flow:**
```
Step 2 Output: Multiple *_stardata_*.h5 files
                    ↓
Step 3: Aggregate and summarize by snapshot/halo
                    ↓
Step 3 Output: Single *_halostarhosts.txt file
``` -->

<!-- **Performance Optimizations:**
- **Batch Processing**: Handles large datasets efficiently using vectorized NumPy operations
- **String Building**: Uses `list.join()` instead of repeated string concatenation for speed
- **Efficient I/O**: Single-pass reading of HDF5 files with minimal memory footprint -->

**Purpose:** This step creates a compact summary of star formation activity that enables Step 4 to efficiently track halo merger histories and assign unique IDs to star-forming halos. 
<!-- The chronological format makes it easy to identify:
- Which halos were actively forming stars at each epoch
- How star formation rates varied over cosmic time  
- Which halos contributed most significantly to stellar mass assembly -->
<!-- 
**Example Usage:**
- Input: 40+ `cptmarvel.cosmo25cmb.4096g5HbwK1BH_stardata_*.h5` files
- Output: `cptmarvel.cosmo25cmb.4096g5HbwK1BH_halostarhosts.txt`
- Result: Timeline of ~100 snapshots showing star formation in ~1000s of halos -->

* Usage: `python writeouthosts_rz.py <sim> <output_dir>`
* Example: `python writeouthosts_rz.py cptmarvel.cosmo25cmb.4096g5HbwK1BH /output/path/`
* Runtime: instant

**Important Notes:**
<!-- - **Halo IDs are Tangos database indices**, not Amiga group IDs (consistent with Step 2)
- Output file size is typically much smaller than input HDF5 files (text vs binary format)
- This summary enables efficient processing in Step 4 without re-reading large HDF5 files -->
- Unbound stars (host ID = -1) are included in the summary for completeness

In [7]:
import writeouthosts_rz

In [8]:
writeouthosts_rz.main(basename, odir=outfile_dir)

Found 42 stardata files
Output file: /home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_halostarhosts.txt


### 3.5) Find and Fill In Main Progenitors

#### Unused

In [ ]:
!tangos serve

In [5]:
tangos.all_simulations()
import tqdm.notebook as tqdm
import halo_trace as ht

In [6]:
# Get all halos in 004096 snapshot
timestep = tangos.get_timestep(ss_dir+"/%4096")
all_halos = timestep.halos.all()
print("There are %d halos in the snapshot." % len(all_halos))

# Filter for halos with stars
halos_with_stars = [h for h in all_halos if h.NStar > 0 and h['Mvir'] > 1e9]

print("\nThere are %d halos with stars." % len(halos_with_stars))
for halo in halos_with_stars:
    print("Halo ID: %s, Mass: %1.2eM☉, Stars: %d" % (halo.halo_number, halo['Mvir'], halo.NStar))

There are 31851 halos in the snapshot.

There are 22 halos with stars.
Halo ID: 1, Mass: 6.41e+10M☉, Stars: 2854124
Halo ID: 2, Mass: 3.25e+10M☉, Stars: 581636
Halo ID: 3, Mass: 2.12e+10M☉, Stars: 143680
Halo ID: 4, Mass: 1.42e+10M☉, Stars: 225480
Halo ID: 5, Mass: 8.38e+09M☉, Stars: 49882
Halo ID: 6, Mass: 7.68e+09M☉, Stars: 20212
Halo ID: 7, Mass: 7.32e+09M☉, Stars: 65167
Halo ID: 8, Mass: 6.97e+09M☉, Stars: 62251
Halo ID: 10, Mass: 6.34e+09M☉, Stars: 2032
Halo ID: 11, Mass: 4.92e+09M☉, Stars: 3257
Halo ID: 13, Mass: 4.78e+09M☉, Stars: 10030
Halo ID: 14, Mass: 3.00e+09M☉, Stars: 5615
Halo ID: 15, Mass: 2.69e+09M☉, Stars: 3
Halo ID: 16, Mass: 2.59e+09M☉, Stars: 1949
Halo ID: 17, Mass: 2.63e+09M☉, Stars: 430
Halo ID: 20, Mass: 8.96e+09M☉, Stars: 14754
Halo ID: 22, Mass: 8.85e+09M☉, Stars: 1081
Halo ID: 23, Mass: 1.63e+09M☉, Stars: 79
Halo ID: 24, Mass: 1.53e+09M☉, Stars: 1470
Halo ID: 28, Mass: 1.21e+09M☉, Stars: 4
Halo ID: 39, Mass: 4.21e+09M☉, Stars: 494
Halo ID: 72, Mass: 2.41e+09M☉

In [7]:
# Get all halos in 004096 snapshot
all_timesteps = tangos.get_simulation(ss_dir).timesteps
halos_stars_dict = {}
for timestep in all_timesteps:
    all_halos = timestep.halos.all()
    # print("There are %d halos in the snapshot." % len(all_halos))

    # Filter for halos with stars
    halos_with_stars = [h for h in all_halos if h.NStar > 0]
    print("Timestep: %s" % timestep.extension[-6:])
    print("There are %d halos with stars.\n" % len(halos_with_stars))
    halos_stars_dict[timestep.extension[-6:]] = halos_with_stars
    # for halo in halos_with_stars:
    #     print("Halo ID: %s, Mass: %1.2eM☉, Stars: %d" % (halo.halo_number, halo['Mvir'], halo.NStar))

Timestep: 000192
There are 55 halos with stars.

Timestep: 000288
There are 57 halos with stars.

Timestep: 000384
There are 59 halos with stars.

Timestep: 000480
There are 57 halos with stars.

Timestep: 000576
There are 54 halos with stars.

Timestep: 000672
There are 55 halos with stars.

Timestep: 000768
There are 51 halos with stars.

Timestep: 000864
There are 50 halos with stars.

Timestep: 000960
There are 51 halos with stars.

Timestep: 001056
There are 49 halos with stars.

Timestep: 001152
There are 50 halos with stars.

Timestep: 001248
There are 49 halos with stars.

Timestep: 001344
There are 50 halos with stars.

Timestep: 001440
There are 48 halos with stars.

Timestep: 001536
There are 47 halos with stars.

Timestep: 001632
There are 46 halos with stars.

Timestep: 001728
There are 47 halos with stars.

Timestep: 001824
There are 45 halos with stars.

Timestep: 001920
There are 45 halos with stars.

Timestep: 002016
There are 46 halos with stars.

Timestep: 002112
The

In [8]:
halos_stars_dict['004096']

[<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_1' | NDM=11956794 Nstar=2854124 Ngas=5544225>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_2' | NDM=6020296 Nstar=581636 Ngas=3055754>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_3' | NDM=4023224 Nstar=143680 Ngas=1610867>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_4' | NDM=2840249 Nstar=225480 Ngas=364281>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_5' | NDM=1674684 Nstar=49882 Ngas=222673>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_6' | NDM=1521044 Nstar=20212 Ngas=279119>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_7' | NDM=1473366 Nstar=65167 Ngas=139001>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096/halo_8' | NDM=1407596 Nstar=62251 Ngas=115365>,
 <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5H

### Parallelization Helpers

In [6]:
import math
import pynbody
from multiprocessing import Pool, Lock, Manager, Queue
import tqdm.notebook as tqdm

progress_queue = None
print_lock = None

def _init_worker(lock):
    """Initialize worker with shared lock"""
    global print_lock
    print_lock = lock

def safe_print(*args, **kwargs):
    """Thread-safe print function"""
    with print_lock:
        print(*args, **kwargs)

def safe_update_pbar(pbar, total):
    """Thread-safe update of progress bar"""
    with print_lock:
        pbar.update(total)
        pbar.refresh()

def create_thread_groups(items, n_threads):
    """
    Split a list of items into groups for parallel processing.
    
    This function divides a list of items (typically simulation timesteps) into 
    roughly equal groups that can be processed by different threads or processes
    in parallel. It ensures efficient load balancing across available threads.
    
    Parameters
    ----------
    items : list or array-like
        Items to be processed (e.g., simulation timesteps, file paths)
    n_threads : int
        Number of threads/processes to use for parallel processing
        
    Returns
    -------
    groups : list of lists
        Each sublist contains items for one thread to process. The number of
        sublists equals min(n_threads, len(items))
    """
    items = list(items)  # Ensure it's a list
    n_items = len(items)
    
    if n_threads >= n_items:
        # If more threads than items, give each item its own thread
        return [[item] for item in items]
    
    # Calculate items per thread
    items_per_thread = math.ceil(n_items / n_threads)
    
    groups = []
    for i in range(0, n_items, items_per_thread):
        group = items[i:i + items_per_thread]
        groups.append(group)
    
    return groups

def _parallelize(func, item, index, progress_bar=None):
    """
    Worker function for parallel processing with progress tracking.
    
    This is an internal helper function used by parallelize_func() to execute
    a given function on a group of items while tracking progress. It handles
    both single items and lists of items.
    
    Parameters
    ----------
    func : callable
        Function to apply to each item. Should accept a single argument.
    item : object or list
        Single item or list of items to process
    index : int
        Index of the worker process (used for progress bar positioning)
    progress_bar : tqdm.tqdm object, optional
        Progress bar instance for tracking completion. If None and item is a list,
        creates a new progress bar.
        
    Returns
    -------
    index : int
        Returns the worker index (used for process identification) when done
    """
    if type(item) is not str:
        if progress_bar:
            global pbar
        if not progress_bar:
            pbar = tqdm.tqdm(total=len(item), position=index)
        for it in item:
            func(it)
            safe_update_pbar(pbar, 1)
        if not progress_bar:
            pbar.close()
    return index

def parallelize_func(func: callable, items, n_threads: int, group_pbar=False):
    """
    Execute a function in parallel across multiple threads with progress tracking.
    
    This function provides a high-level interface for parallel processing of items
    using multiprocessing.Pool. It automatically handles thread group creation,
    progress bar management, and process coordination.
    
    Parameters
    ----------
    func : callable
        Function to apply to each item. Must accept a single argument.
        Should be picklable for multiprocessing.
    items : list or array-like
        Collection of items to process (e.g., file paths, timesteps)
    n_threads : int
        Number of parallel processes to use
    group_pbar : bool, optional
        If True, shows one progress bar rather than individual ones per thread.
        Default is False.
        
    Returns
    -------
    None
    """
    groups = create_thread_groups(items, n_threads)
    lock = Lock()
    if group_pbar:
        global pbar
        pbar = tqdm.tqdm(total=len(groups))
        with Pool(processes=len(groups), initializer=_init_worker, initargs=(lock,)) as pool:
            pool.starmap(_parallelize, [(func, group, i, True) for i, group in enumerate(groups)])
    else:
        with Pool(processes=n_threads, initializer=_init_worker, initargs=(lock,)) as pool:
            pool.starmap(_parallelize, [(func, group, i, None) for i, group in enumerate(groups)])

def _process_step(step):
    """
    Process a single simulation timestep to create amiga.grp halo grouping files.
    
    This function loads a simulation snapshot, identifies halos using pynbody's
    halo finder, and writes the halo group information to an amiga.grp file.
    
    Parameters
    ----------
    step : str
        Timestep identifier/extension (e.g., '004096')
        Used to construct the full file path: sim_base + step
        
    Returns
    -------
    None
        Function operates by side effect, writing amiga.grp files
    """
    path = sim_base + step
    # print(path)
    # time.sleep(1)
    safe_print('Loading <{}>'.format(step))
    f = pynbody.load(path)
    safe_print('  Loading halos for <{}>'.format(step))
    try:
        h = f.halos(halo_numbers='v1')
        safe_print('    Found halos for <{}>: {}'.format(step, h))
        # safe_print('    Writing amiga.grp for <{}>'.format(step))
        # f['amiga.grp'] = h.get_group_array()
        # f['amiga.grp'].write(overwrite=True)
        # safe_print('    Finished writing amiga.grp for <{}>'.format(step))
    except Exception as e:
        safe_print('  ERROR loading halos for <{}>: {}'.format(step, e))


In [7]:
import math
import time
import pynbody
from multiprocessing import Pool, Lock
import tqdm.notebook as tqdm # Using notebook-compatible tqdm

# This should be defined in your main script execution block
# sim_base = "/path/to/your/simulation/data/snapshot_" 

# Global lock for safe printing in the multi-pbar case
print_lock = None

def _init_worker_lock(lock):
    """Makes the lock global for use in worker processes."""
    global print_lock
    print_lock = lock

def safe_print(*args, **kwargs):
    """Thread-safe print function, requires the lock to be initialized."""
    with print_lock:
        print(*args, **kwargs)

def create_thread_groups(items, n_threads):
    """
    Splits a list of items into groups for parallel processing. 
    (Your implementation is correct and retained).
    """
    items = list(items)
    n_items = len(items)
    
    if n_threads >= n_items:
        return [[item] for item in items]
    
    items_per_thread = math.ceil(n_items / n_threads)
    groups = []
    for i in range(0, n_items, items_per_thread):
        group = items[i:i + items_per_thread]
        groups.append(group)
    
    return groups

def worker_task_single_pbar(args):
    """
    Worker for the single-pbar case.
    It applies a function to a group of items and returns the number processed.
    It does NOT touch any progress bar.
    """
    func, item_group = args
    results = []
    for item in item_group:
        results.append(func(item))
    return len(item_group) # Report how many items were processed

def worker_task_multi_pbar(args):
    """
    Worker for the multi-pbar case.
    It creates and manages its own progress bar for its assigned items.
    """
    func, item_group, index, lock = args
    _init_worker_lock(lock) # Initialize lock for this worker
    
    with tqdm.tqdm(total=len(item_group), position=index, desc=f"Process {index+1}") as pbar:
        for item in item_group:
            func(item)
            pbar.update(1)

def parallelize_func(func: callable, items, n_threads: int, group_pbar: bool = False):
    """
    Executes a function in parallel with a choice of progress bar display.

    Parameters
    ----------
    func : callable
        Function to apply to each item. Must be picklable.
    items : list
        A list of items to process.
    n_threads : int
        Number of parallel processes to use.
    group_pbar : bool
        If True (default), shows one progress bar for the entire task.
        If False, shows a separate progress bar for each process.
    """
    groups = create_thread_groups(items, n_threads)
    
    if group_pbar:
        # --- SINGLE, CENTRALIZED PBAR (RACE-CONDITION-FREE) ---
        with Pool(processes=n_threads) as pool:
            # Create the progress bar in the main process
            with tqdm.tqdm(total=len(items), desc="Overall Progress") as pbar:
                # pool.imap_unordered processes items and yields results as they complete
                # This allows for real-time updates to the progress bar
                args_list = [(func, g) for g in groups]
                for num_processed in pool.imap_unordered(worker_task_single_pbar, args_list):
                    pbar.update(num_processed)
    else:
        # --- MULTIPLE PBARS (ONE PER PROCESS) ---
        lock = Lock()
        with Pool(processes=len(groups)) as pool:
            args_list = [(func, group, i, lock) for i, group in enumerate(groups)]
            pool.map(worker_task_multi_pbar, args_list)

# --- Your Specific Processing Function ---
def _process_step(step):
    """
    Processes a single simulation timestep.
    NOTE: `sim_base` must be a global variable or passed differently.
          Making it global is simplest for multiprocessing if it's read-only.
    """
    try:
        path = sim_base + step
        # Use safe_print if you need console output during processing
        # safe_print(f'Loading <{step}>') 
        f = pynbody.load(path)
        h = f.halos(halo_numbers='v1')
        # f['amiga.grp'] = h.get_group_array()
        # f['amiga.grp'].write(overwrite=True)
    except Exception as e:
        # It's safer to print errors to see what failed
        print(f'ERROR processing step <{step}>: {e}')
    return step # Return something to confirm completion


In [8]:
steps = [step.extension for step in tangos.get_simulation("cptmarvel.4096g5HbwK1BH_bn").timesteps]

In [11]:
parallelize_func(_process_step, steps, n_threads=8, group_pbar=False)

In [9]:
# !tangos serve

### NEW

In [5]:
# Get all halos in 004096 snapshot
timestep = tangos.get_timestep(ss_dir+"/%4096")
all_halos = timestep.halos.all()
print("There are %d halos in the snapshot." % len(all_halos))

# Filter for halos with stars
halos_with_stars = [h for h in all_halos if h.NStar > 0]

There are 31851 halos in the snapshot.


In [6]:
import warnings
from sqlalchemy.exc import SAWarning
import tangos.examples.mergers as mergers
import pandas as pd

warnings.filterwarnings("ignore", category=SAWarning)

for halo in tqdm.tqdm(halos_with_stars):
    datab = pd.DataFrame(columns=["snapshot", "time_gyr", "main_halo_num", "merging_halo_num", "main_mstar", "merging_mstar", "main_mvir", "merging_mvir", "main_mgas", "merging_mgas", "redshift", "main_haloid", "merging_haloid"])
    save_path = outfile_dir + 'mergers/' + halo.timestep.extension + '_' + str(halo.halo_number) + '.csv'
    # print("\nMain Progenitor z=0 Halo ID: %s" % (halo.halo_number))
    redshift, ratio, progenitor_halos = mergers.get_mergers_of_major_progenitor(halo)
    # each item of progenitor_halos is a pair; the first is the major progenitor, the second is the thing merging into it
    progenitor_halos = [x for x in progenitor_halos if x[1].NStar > 0 and x[0].NStar > 0]
    print(progenitor_halos)
    merging_structures = [x[1] for x in progenitor_halos if x[1].NStar > 0]
    main_structures = [x[0] for x in progenitor_halos if x[0].NStar > 0]
    if len(merging_structures) == 0:
        continue
    else:
        print("\nMain Progenitor z=0 Halo ID: %s" % (halo.halo_number))
        print("There are %d mergers into the major progenitor branch.\n" % len(merging_structures))

        for i, merging_halo in enumerate(merging_structures):
            snapshot = merging_halo.timestep.extension[-6:]
            print("Merging Halo %d/%d at Snapshot %s" % (i+1, len(merging_structures), snapshot))
            try:
                merging_mvir = merging_halo['Mvir']
            except:
                merging_mvir = np.nan
            try:
                main_mvir = progenitor_halos[i][0]['Mvir']
            except:
                main_mvir = np.nan
            row = [snapshot, merging_halo.timestep.time_gyr, 
                   main_structures[i].halo_number, merging_halo.halo_number, 
                   main_structures[i]['M_star'], merging_halo['M_star'], 
                   main_mvir, merging_mvir, 
                   main_structures[i]['M_gas'], merging_halo['M_gas'], 
                   merging_halo.timestep.redshift,
                   main_structures[i].id, merging_halo.id]

            datab.loc[len(datab)] = row

            merging_halo_num = merging_halo.halo_number
            progenitor_halo_num = main_structures[i].halo_number
            # print(merging_halo.keys())
            # print("  Mass: %f" % merging_halo['Mvir'])
            print("  Merging Halo ID: %s, NStars: %d, NGas: %d" % (merging_halo_num, merging_halo.NStar, merging_halo.NGas))
            print("  Main Progenitor Halo ID: %s, NStars: %d, NGas: %d" % (progenitor_halo_num, main_structures[i].NStar, main_structures[i].NGas))
            # print("  Redshift: %f" % redshift[i])
            # print("  Ratio: %f" % ratio[i])
            print()
        # sort by time_gyr
        datab = datab.sort_values(by="time_gyr",ignore_index=True)
        print("Saving to %s" % save_path)
        datab.to_csv(save_path, index=False)
        print(datab)

  0%|          | 0/51 [00:00<?, ?it/s]

[(<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002400/halo_1' | NDM=9986073 Nstar=1301624 Ngas=4580348>, <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002400/halo_29' | NDM=100541 Nstar=20920 Ngas=136226>), (<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002208/halo_1' | NDM=10453917 Nstar=1213914 Ngas=4728877>, <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002208/halo_27' | NDM=133898 Nstar=19200 Ngas=136708>), (<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002208/halo_1' | NDM=10453917 Nstar=1213914 Ngas=4728877>, <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002208/halo_30' | NDM=242235 Nstar=2120 Ngas=13687>), (<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002112/halo_1' | NDM=10132040 Nstar=1016614 Ngas=4760274>, <Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002112/halo_11' | NDM=354131 Nstar=69012 Ngas=507662>), (<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.40

In [7]:
len(merging_structures)

0

In [8]:
merging_halo.all_properties

[<HaloProperty Mvir=2.40e+07 of <Halo 668 of ...>>,
 <HaloProperty Rvir=8.10e-01 of <Halo 668 of ...>>,
 <HaloProperty Xc=1.01e+04 of <Halo 668 of ...>>,
 <HaloProperty Yc=1.11e+04 of <Halo 668 of ...>>,
 <HaloProperty Zc=9.03e+03 of <Halo 668 of ...>>,
 <HaloProperty VXc=1.12e+02 of <Halo 668 of ...>>,
 <HaloProperty VYc=1.02e+02 of <Halo 668 of ...>>,
 <HaloProperty VZc=3.90e-01 of <Halo 668 of ...>>,
 <HaloProperty Vmax=1.38e+01 of <Halo 668 of ...>>,
 <HaloProperty fMhires=1.00e+00 of <Halo 668 of ...>>,
 <HaloProperty M_gas=0 of <Halo 668 of ...>>,
 <HaloProperty M_star=1.32e+04 of <Halo 668 of ...>>,
 <HaloProperty n_gas=0 of <Halo 668 of ...>>,
 <HaloProperty n_star=69 of <Halo 668 of ...>>,
 <HaloProperty n_dm=4937 of <Halo 668 of ...>>,
 <HaloProperty npart=5006 of <Halo 668 of ...>>]

In [9]:
progenitor_halos[i][0].timestep

IndexError: list index out of range

In [ ]:
halodict['000384'][0].timestep.redshift

NameError: name 'halodict' is not defined

In [ ]:
import warnings
from sqlalchemy.exc import SAWarning

warnings.filterwarnings("ignore", category=SAWarning)

halodict = {}

def safe_add(dict, key, value):
    if key not in dict:
        dict[key] = [value]
    else:
        dict[key].append(value)
    return dict

for halo in tqdm.tqdm(halos_with_stars):
    import tangos.examples.mergers as mergers
    # print("\nMain Progenitor z=0 Halo ID: %s" % (halo.halo_number))
    redshift, ratio, progenitor_halos = mergers.get_mergers_of_major_progenitor(halo)
    # each item of progenitor_halos is a pair; the first is the major progenitor, the second is the thing merging into it
    merging_structures = [x[1] for x in progenitor_halos if x[1].NStar > 0]
    if len(merging_structures) == 0:
        pass
        # print("\nMain Progenitor z=0 Halo ID: %s" % (halo.halo_number))
        # print("There are no mergers into the major progenitor branch.\n")
    else:
        print("\nMain Progenitor z=0 Halo ID: %s" % (halo.halo_number))
        print("There are %d mergers into the major progenitor branch.\n" % len(merging_structures))

        for i, merging_halo in enumerate(merging_structures):
            snapshot = merging_halo.timestep.extension[-6:]
            print("Merging Halo %d/%d at Snapshot %s" % (i+1, len(merging_structures), snapshot))
            halodict = safe_add(halodict, snapshot, merging_halo)
            
            merging_halo_num = merging_halo.halo_number
            progenitor_halo_num = progenitor_halos[i][0].halo_number
            # print(merging_halo.keys())
            # print("  Mass: %f" % merging_halo['Mvir'])
            print("  Merging Halo ID: %s, NStars: %d, NGas: %d" % (merging_halo_num, merging_halo.NStar, merging_halo.NGas))
            print("  Main Progenitor Halo ID: %s, NStars: %d, NGas: %d" % (progenitor_halo_num, progenitor_halos[i][0].NStar, progenitor_halos[i][0].NGas))
            # print("  Redshift: %f" % redshift[i])
            # print("  Ratio: %f" % ratio[i])
            print()

# for halo in tqdm.tqdm(halos_with_stars):
#     pass
    # print_main_progenitor_info(halo)

  0%|          | 0/51 [00:00<?, ?it/s]


Main Progenitor z=0 Halo ID: 1
There are 20 mergers into the major progenitor branch.

Merging Halo 1/20 at Snapshot 002400
  Merging Halo ID: 29, NStars: 20920, NGas: 136226
  Main Progenitor Halo ID: 1, NStars: 1301624, NGas: 4580348

Merging Halo 2/20 at Snapshot 002208
  Merging Halo ID: 27, NStars: 19200, NGas: 136708
  Main Progenitor Halo ID: 1, NStars: 1213914, NGas: 4728877

Merging Halo 3/20 at Snapshot 002208
  Merging Halo ID: 30, NStars: 2120, NGas: 13687
  Main Progenitor Halo ID: 1, NStars: 1213914, NGas: 4728877

Merging Halo 4/20 at Snapshot 002112
  Merging Halo ID: 11, NStars: 69012, NGas: 507662
  Main Progenitor Halo ID: 1, NStars: 1016614, NGas: 4760274

Merging Halo 5/20 at Snapshot 002016
  Merging Halo ID: 6, NStars: 76265, NGas: 743071
  Main Progenitor Halo ID: 1, NStars: 945775, NGas: 4658347

Merging Halo 6/20 at Snapshot 001728
  Merging Halo ID: 3, NStars: 728659, NGas: 622659
  Main Progenitor Halo ID: 1, NStars: 772591, NGas: 3115839

Merging Halo 7/20

In [ ]:
halodict.keys()

dict_keys(['002400', '002208', '002112', '002016', '001728', '001632', '001344', '000960', '000864', '000768', '000672', '000576', '000384', '003744', '003648', '003552', '003456', '003360', '003264', '003168', '002976', '002880', '002784', '002592', '002496', '001920', '001536', '001440', '001248', '001152', '000192', '002688', '000480', '002304', '001056', '000288', '003072', '003840'])

In [ ]:
all_timesteps = tangos.get_simulation(ss_dir).timesteps
all_timesteps = [ts.extension[-6:] for ts in all_timesteps]
tsteps_used = list(halodict.keys())

# maxim = '001408'
# select_timesteps = [ts for ts in all_timesteps if int(ts) <= int(maxim)]
# select_timesteps

In [ ]:
# sort tsteps_used by int(i) for i in tsteps_used, reversed
tsteps_used = sorted(tsteps_used, key=int, reverse=True)
tsteps_used

['003840',
 '003744',
 '003648',
 '003552',
 '003456',
 '003360',
 '003264',
 '003168',
 '003072',
 '002976',
 '002880',
 '002784',
 '002688',
 '002592',
 '002496',
 '002400',
 '002304',
 '002208',
 '002112',
 '002016',
 '001920',
 '001728',
 '001632',
 '001536',
 '001440',
 '001344',
 '001248',
 '001152',
 '001056',
 '000960',
 '000864',
 '000768',
 '000672',
 '000576',
 '000480',
 '000384',
 '000288',
 '000192']

### Halo Trace (Elaad)

In [9]:
# parallelize_func(print_main_progenitor_info, all_halos, n_threads=20, group_pbar=False)

In [ ]:
# sim_base = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/snapshots_200crit_cptmarvel'
# sim_base = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn'
sim_base = simpath + ss_dir + '/'
# ahf_dir = None
# ahf_dir = 'ahf_200'
# grplist = None

In [19]:
def trace_step(ts):
    print(ts)
    grplist = np.array([halo.halo_number for halo in halodict[ts]])
    print(grplist)
    steplist = [int(tstep) for tstep in all_timesteps if int(tstep) <= int(ts)]
    print(steplist)
    trace = ht.tracing.trace_halos(sim_base=sim_base,
                                   grplist=grplist,
                                   steplist=steplist)
    return 0

In [26]:
for i in tqdm.tqdm(tsteps_used):
    trace_step(i)

  0%|          | 0/38 [00:00<?, ?it/s]

003840
[24]
[192, 288, 384, 480, 576, 672, 768, 864, 960, 1056, 1152, 1248, 1344, 1440, 1536, 1632, 1728, 1824, 1920, 2016, 2112, 2208, 2304, 2400, 2496, 2592, 2688, 2784, 2880, 2976, 3072, 3168, 3264, 3360, 3456, 3552, 3648, 3744, 3840]
Steplist ['/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003840', '/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003744', '/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003648', '/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003552', '/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003456', '/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003360', '/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003264', '/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003168', '/home/ns1917/tangos_sims/storm.4096g5Hb

In [13]:
filenames = glob.glob(sim_base + '/*merge.hdf5')
for file in filenames:
    print(file)
    trace = pd.read_hdf(file, key='ids')
    # Fix the index by removing the '004096' prefix
    print("Original index:", trace.index.name)
    # trace.index = trace.index.str.replace(trace.index.name, '')
    # Convert to integers if desired
    trace.index = trace.index.astype(int)
    print(trace.head())
    # trace.to_hdf(file, key='ids')

/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003840.trace_back_merge.hdf5
Original index: 003840
        003744  003648  003552  003456  003360  003264  003168  003072  \
003840                                                                   
24          28      29      39      31      35      35      37      36   

        002976  002880  ...  001056  000960  000864  000768  000672  000576  \
003840                  ...                                                   
24          37      38  ...      73      75      70      71      82      82   

        000480  000384  000288  000192  
003840                                  
24         108      91      88      46  

[1 rows x 38 columns]
/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003744.trace_back_merge.hdf5
Original index: 003744
        003648  003552  003456  003360  003264  003168  003072  002976  \
003744                                                   

In [14]:
import pandas as pd
from tangos import get_timestep, core
from tangos.core import get_default_session as session

def create_phantom(ts, halo_number):
    # Make a new Halo object with a special flag or negative halo number
    phantom = core.Halo(
        halo_number=halo_number,
        timestep=ts,
        finder_id=-halo_number,  # Use negative to distinguish phantom
        # Add other properties as needed
    )
    session.add(phantom)
    session.commit()
    return phantom

def check_and_fix_links_with_phantoms(df, sim_prefix="sim", dry_run=True, create_phantoms=False):
    for chain_idx, row in df.iterrows():
        halos = []
        for snapshot, halo_number in row.items():
            if pd.isnull(halo_number):
                halos.append(None)
            else:
                ts = get_timestep(f"{sim_prefix}/%{snapshot}")
                halo = ts.halos.filter_by(halo_number=int(halo_number)).first() if ts else None
                halos.append(halo)
        for i in range(1, len(halos)):
            descendant = halos[i-1]
            progenitor = halos[i]
            # If progenitor missing from DB and create_phantoms is True, create phantom
            if progenitor is None and not pd.isnull(row.iloc[i]):
                ts = get_timestep(f"{sim_prefix}/{row.index[i]}")
                if create_phantoms and ts is not None:
                    print(f"[CHAIN {chain_idx}] Creating PHANTOM for halo {row.iloc[i]} in snapshot {row.index[i]}")
                    progenitor = create_phantom(ts, int(row.iloc[i]))
                    halos[i] = progenitor
            if descendant is None or progenitor is None:
                continue
            found = False
            for link in descendant.previous:
                if link.halo_from == progenitor:
                    found = True
                    break
            if not found:
                print(f"[CHAIN {chain_idx}] Missing link: {descendant} (halo_number={descendant.halo_number}, snapshot={descendant.timestep.extension}) "
                      f"should have progenitor {progenitor} (halo_number={progenitor.halo_number}, snapshot={progenitor.timestep.extension})")
                if not dry_run:
                    new_link = core.HaloLink(
                        halo_from=progenitor,
                        halo_to=descendant,
                        type='progenitor',
                        weight=1.0
                    )
                    session.add(new_link)
                    print(f"  -> Link ADDED")
        if not dry_run:
            session.commit()
            print(f"[CHAIN {chain_idx}] Committed new links to DB.")


In [15]:
def get_halo(snapshot, halo_number):
    ts = tangos.get_timestep(f"{ss_dir}/%{snapshot}")
    # print(f"Retrieved timestep: {ts}")
    return ts.halos.filter_by(halo_number=int(halo_number)).first()

In [16]:
def process_halo_data(filename):
    df = pd.read_hdf(filename, key='ids')
    snapshot = df.index.name
    # rows = df.values
    print("Snapshot:", snapshot)
    for index, row in df.iterrows():
        # print(index)
        halo_numbers_df = row.values.tolist()
        halo_numbers_df.insert(0, index)
        snapshots_df = row.index.values.tolist()
        snapshots_df.insert(0, snapshot)
        unique_id_strings = []
        for halo_num, snap in zip(halo_numbers_df, snapshots_df):
            unique_id_string = f"{snap}_{halo_num}"
            unique_id_strings.append(unique_id_string)
        print(f"    Unique IDs: {unique_id_strings}")
        # print(halo_numbers_df)
        # print(snapshots_df)
        halo = get_halo(snapshot, index)
        print(f"  Retrieved halo: {halo.halo_number}")

        unique_id_tangos = []
        halo_numbers, dbids = halo.calculate_for_progenitors('halo_number()','dbid()')
        snapshots = [tangos.get_halo(dbid).timestep.extension[-6:] for dbid in dbids]
        print(f"    Retrieved progenitor halos: {halo_numbers}")
        # print(f"    Retrieved progenitor snapshots: {snapshots}")
        for halo_num, snap in zip(halo_numbers, snapshots):
            unique_id_string = f"{snap}_{halo_num}"
            unique_id_tangos.append(unique_id_string)
        # print(f"    Unique IDs: {unique_id_tangos}")

        # check if each one is in the tangos database
        missing_links = False
        for unique_id in unique_id_strings:
            if unique_id not in unique_id_tangos and not unique_id.endswith('-1') and not unique_id.endswith('nan'):
                missing_links = True
                print(f"    ID {unique_id} not found in tangos links")

        if missing_links:
            return unique_id_strings
        # for unique_id in unique_id_tangos:
        #     if unique_id not in unique_id_strings:
        #         print(f"    ID {unique_id} not found in halo links")

    return None

In [17]:
halos_missing_links = []
for filename in filenames:
    links = process_halo_data(filename)
    if links is not None:
        halos_missing_links.append(links)

Snapshot: 003840
    Unique IDs: ['003840_24', '003744_28', '003648_29', '003552_39', '003456_31', '003360_35', '003264_35', '003168_37', '003072_36', '002976_37', '002880_38', '002784_38', '002688_34', '002592_35', '002496_36', '002400_38', '002304_39', '002208_40', '002112_41', '002016_40', '001920_41', '001824_41', '001728_40', '001632_37', '001536_40', '001440_40', '001344_43', '001248_50', '001152_66', '001056_73', '000960_75', '000864_70', '000768_71', '000672_82', '000576_82', '000480_108', '000384_91', '000288_88', '000192_46']
  Retrieved halo: 24
    Retrieved progenitor halos: [ 24  28  29  39  31  35  35  37  36  37  38  38  34  35  36  38  39  40
  41  40  41  41  40  37  40  40  43  50  66  73  75  70  71  82  82 108
  91  88  46]
Snapshot: 003744
    Unique IDs: ['003744_13', '003648_13', '003552_12', '003456_12', '003360_12', '003264_11', '003168_11', '003072_11', '002976_11', '002880_11', '002784_11', '002688_11', '002592_11', '002496_11', '002400_8', '002304_6', '0022

In [18]:
halos_missing_links

[['003168_27',
  '003072_-1',
  '002976_36',
  '002880_26',
  '002784_25',
  '002688_43',
  '002592_25',
  '002496_15',
  '002400_13',
  '002304_11',
  '002208_12',
  '002112_15',
  '002016_15',
  '001920_15',
  '001824_15',
  '001728_15',
  '001632_15',
  '001536_15',
  '001440_16',
  '001344_15',
  '001248_14',
  '001152_14',
  '001056_13',
  '000960_14',
  '000864_19',
  '000768_18',
  '000672_20',
  '000576_17',
  '000480_16',
  '000384_12',
  '000288_21',
  '000192_63'],
 ['002016_6',
  '001920_7',
  '001824_14',
  '001728_-1',
  '001632_6',
  '001536_6',
  '001440_6',
  '001344_6',
  '001248_6',
  '001152_5',
  '001056_5',
  '000960_5',
  '000864_6',
  '000768_5',
  '000672_8',
  '000576_15',
  '000480_22',
  '000384_33',
  '000288_30',
  '000192_20'],
 ['001536_25',
  '001440_-1',
  '001344_29',
  '001248_29',
  '001152_24',
  '001056_22',
  '000960_29',
  '000864_31',
  '000768_28',
  '000672_28',
  '000576_32',
  '000480_37',
  '000384_35',
  '000288_50',
  '000192_70']]

In [19]:
halos_missing_links[0]

['003168_27',
 '003072_-1',
 '002976_36',
 '002880_26',
 '002784_25',
 '002688_43',
 '002592_25',
 '002496_15',
 '002400_13',
 '002304_11',
 '002208_12',
 '002112_15',
 '002016_15',
 '001920_15',
 '001824_15',
 '001728_15',
 '001632_15',
 '001536_15',
 '001440_16',
 '001344_15',
 '001248_14',
 '001152_14',
 '001056_13',
 '000960_14',
 '000864_19',
 '000768_18',
 '000672_20',
 '000576_17',
 '000480_16',
 '000384_12',
 '000288_21',
 '000192_63']

In [20]:
row = 0
for i in range(len(halos_missing_links[row])):
    # we find the missing link
    if halos_missing_links[row][i].endswith('-1'):
        # check lengths
        if i > 0 and i < len(halos_missing_links[row])-1:
            prev_snap, prev_halonum = halos_missing_links[row][i+1].split('_')
            prev_halo = get_halo(prev_snap, int(prev_halonum))
            print(prev_halo.links)
            next_snap, next_halonum = halos_missing_links[row][i-1].split('_')
            next_halo = get_halo(next_snap, int(next_halonum))
            cur_snap = halos_missing_links[row][i].split('_')[0]
            print(f"Previous: {prev_snap}_{prev_halonum}, Current: {cur_snap}_{-1}, Next: {next_snap}_{next_halonum}")

            # make phantom halo
            ph_halonum = len(tangos.get_timestep(f"{ss_dir}/%{cur_snap}").phantoms.all()) + 1
            cur_halo = tangos.core.halo.PhantomHalo(
                halo_number=ph_halonum,
                timestep=prev_halo.timestep.next,
                finder_id=-ph_halonum
            )
            tangos.core.get_default_session().add(cur_halo)

            # make link from prev_halo to cur_halo
            link1 = tangos.core.HaloLink(
                halo_to=prev_halo,
                halo_from=cur_halo,
                relationship=tangos.core.get_or_create_dictionary_item(tangos.core.get_default_session(), 'progenitor'),
                weight=2
            )
            link2 = tangos.core.HaloLink(
                halo_to=cur_halo,
                halo_from=prev_halo,
                relationship=tangos.core.get_or_create_dictionary_item(tangos.core.get_default_session(), 'descendant'),
                weight=2
            )
            link3 = tangos.core.HaloLink(
                halo_from=cur_halo,
                halo_to=next_halo,
                relationship=tangos.core.get_or_create_dictionary_item(tangos.core.get_default_session(), 'descendant'),
                weight=2
            )
            link4 = tangos.core.HaloLink(
                halo_to=cur_halo,
                halo_from=next_halo,
                relationship=tangos.core.get_or_create_dictionary_item(tangos.core.get_default_session(), 'progenitor'),
                weight=2
            )
            tangos.core.get_default_session().add_all([link1, link2, link3, link4])
            tangos.core.get_default_session().commit()
            print(f"Created phantom halo: {cur_halo}")

        # halo = get_halo(links.split('_')[0], int(links.split('_')[1]))
        # print(f"Missing link for halo {halo.halo_number} in snapshot {halo.timestep.extension[-6:]}")
    # halo.previous

SELECT halolink.id AS halolink_id, halolink.halo_from_id AS halolink_halo_from_id, halolink.halo_to_id AS halolink_halo_to_id, halolink.weight AS halolink_weight, halolink.creator_id AS halolink_creator_id, halolink.relation_id AS halolink_relation_id 
FROM halolink 
WHERE halolink.halo_from_id = ?
Previous: 002976_36, Current: 003072_-1, Next: 003168_27
Created phantom halo: <PhantomHalo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003072/phantom_9' | NDM=0 Nstar=0 Ngas=0>


In [21]:
prev_halo.next

<PhantomHalo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003072/phantom_9' | NDM=0 Nstar=0 Ngas=0>

In [22]:
cur_halo.next

<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003168/halo_27' | NDM=90827 Nstar=33652 Ngas=162680>

In [23]:
next_halo.previous

<PhantomHalo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003072/phantom_9' | NDM=0 Nstar=0 Ngas=0>

In [24]:
remove_halo = get_halo('003072', 2)

In [25]:
remove_halo.next

<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003168/halo_2' | NDM=6136514 Nstar=429886 Ngas=3012674>

In [ ]:
!tangos serve

Starting server in PID 3317257.
2025-08-11 18:38:06,371 INFO  [waitress:449][MainThread] Serving on http://[::1]:6543
2025-08-11 18:38:06,371 INFO  [waitress:449][MainThread] Serving on http://127.0.0.1:6543
2025-08-11 18:39:07,987 : Tree build complete; total time 3.56s
2025-08-11 18:39:07,987 INFO  [tangos.log:72][waitress-1] Tree build complete; total time 3.56s
2025-08-11 18:39:07,988 :   Progenitor query took 1.89s
2025-08-11 18:39:07,988 INFO  [tangos.log:73][waitress-1]   Progenitor query took 1.89s
2025-08-11 18:39:07,988 :   Property query took 0.02s
2025-08-11 18:39:07,988 INFO  [tangos.log:74][waitress-1]   Property query took 0.02s
2025-08-11 18:39:07,988 :   Tree post-processing took 1.65s
2025-08-11 18:39:07,988 INFO  [tangos.log:75][waitress-1]   Tree post-processing took 1.65s


In [83]:
from tangos.core import HaloLink, get_default_session

session = get_default_session()

# Find the link you want to remove. For example, between halo_a and halo_b with a certain relationship:
link = session.query(HaloLink).filter_by(halo_from=prev_halo, halo_to=remove_halo).first()
link = session.query(HaloLink).filter_by(halo_from=remove_halo, halo_to=prev_halo).first()
link = session.query(HaloLink).filter_by(halo_from=next_halo, halo_to=remove_halo).first()
link = session.query(HaloLink).filter_by(halo_from=remove_halo, halo_to=next_halo).first()
print(link)

None


In [72]:
session.refresh(prev_halo)
session.refresh(next_halo)
session.refresh(cur_halo)  # The phantom

In [55]:
!tangos serve

Starting server in PID 2691768.
2025-08-05 13:53:12,419 INFO  [waitress:449][MainThread] Serving on http://[::1]:6543
2025-08-05 13:53:12,419 INFO  [waitress:449][MainThread] Serving on http://127.0.0.1:6543
2025-08-05 13:54:15,144 : Tree build complete; total time 0.28s
2025-08-05 13:54:15,144 INFO  [tangos.log:72][waitress-2] Tree build complete; total time 0.28s
2025-08-05 13:54:15,145 :   Progenitor query took 0.19s
2025-08-05 13:54:15,145 INFO  [tangos.log:73][waitress-2]   Progenitor query took 0.19s
2025-08-05 13:54:15,145 :   Property query took 0.02s
2025-08-05 13:54:15,145 INFO  [tangos.log:74][waitress-2]   Property query took 0.02s
2025-08-05 13:54:15,145 :   Tree post-processing took 0.07s
2025-08-05 13:54:15,145 INFO  [tangos.log:75][waitress-2]   Tree post-processing took 0.07s
^C


In [54]:
session.commit()

In [52]:
session.delete(link)

In [56]:
tangos.core.get_default_session().rollback()

In [82]:
!tangos serve

Starting server in PID 2645705.
2025-08-04 18:28:51,678 INFO  [waitress:449][MainThread] Serving on http://[::1]:6543
2025-08-04 18:28:51,678 INFO  [waitress:449][MainThread] Serving on http://127.0.0.1:6543
2025-08-04 18:29:02,974 : Tree build complete; total time 0.47s
2025-08-04 18:29:02,974 INFO  [tangos.log:72][waitress-1] Tree build complete; total time 0.47s
2025-08-04 18:29:02,975 :   Progenitor query took 0.33s
2025-08-04 18:29:02,975 INFO  [tangos.log:73][waitress-1]   Progenitor query took 0.33s
2025-08-04 18:29:02,975 :   Property query took 0.02s
2025-08-04 18:29:02,975 INFO  [tangos.log:74][waitress-1]   Property query took 0.02s
2025-08-04 18:29:02,975 :   Tree post-processing took 0.12s
2025-08-04 18:29:02,975 INFO  [tangos.log:75][waitress-1]   Tree post-processing took 0.12s
^C


In [77]:
!tangos recent-runs 5

************************************************************
Run ID =  19
Command line =  /home/ns1917/anaconda3/envs/dwarf/lib/python3.11/site-packages/ipykernel_launcher.py --f=/run/user/1924706/jupyter/runtime/kernel-v37af4353b09596b0f1d1f14a07c8dcf8801489083.json
Host =  emu
Username =  ns1917
Time =  04/08/25 18:00
>>>    2 halos
************************************************************
Run ID =  18
Command line =  /home/christenc/anaconda3/bin/tangos add elektra.4096g5HbwK1BH_bn --min-particles 10
Host =  emu
Username =  christenc
Time =  03/08/25 23:54
>>>    1 simulations
>>>    40 timesteps
>>>    1336902 halos
>>>    9 simulation properties
************************************************************
Run ID =  17
Command line =  /home/christenc/anaconda3/envs/conda-env-py312/bin/tangos write shrink_center --include-only=n_dm>1000 --for storm.4096g5HbwK1BH_bn
Host =  emu
Username =  christenc
Time =  30/07/25 09:09
>>>    29750 halo properties
******************************

In [46]:
prev_halo.timestep.previous

<TimeStep 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.003072' z=0.31 t=10.30 Gyr>

In [31]:
tangos.get_simulation("storm.4096g5HbwK1BH_bn").timesteps[0].phantoms.all()

[]

In [21]:
df = pd.read_hdf(filenames[0], key='ids')
# Set create_phantoms=True if you want to insert placeholder halos for missing ones
check_and_fix_links_with_phantoms(df, sim_prefix=ss_dir, dry_run=True, create_phantoms=False)

KeyError: 0

In [ ]:
trace = pd.read_hdf('/home/selvani/MAP/pynbody/cptmarvel.cosmo25cmb.4096g5HbwK1BH.004096.trace_back.hdf5', key='ids')
# Fix the index by removing the '004096' prefix
trace.index = trace.index.str.replace('004096', '')

# Convert to integers if desired
trace.index = trace.index.astype(int)

#### Unused

In [ ]:
main_branch = halo.calculate_for_progenitors()
print(main_branch)

In [ ]:
main_branch = []
halo_current = halo

# Step 1: Walk the main progenitor branch
while True:
    main_branch.append(halo_current)
    if len(halo_current.progenitors)==0:
        break
    # The main progenitor is usually the most massive
    halo_current = max(halo_current.progenitors, key=lambda h: getattr(h, 'Mvir', 0))

# Step 2: For each snapshot (along main branch), find all progenitors
for h in main_branch:
    all_progs = h.progenitors
    if not all_progs:
        continue
    # The main progenitor (by definition, on the main branch)
    main_prog = max(all_progs, key=lambda h: getattr(h, 'Mvir', 0))
    # Others are mergers into the main branch at this step
    mergers = [prog for prog in all_progs if prog != main_prog]
    
    print(f"Snapshot: {h.timestep.extension} Halo number: {h.halo_number}")
    print(f"  Main progenitor: {main_prog.halo_number if main_prog else 'N/A'}")
    print(f"  Mergers at this step: {[m.halo_number for m in mergers]}")

### 4) IDUniqueHost_rz.py

Step 4 of stellar halo pipeline                                                                                                                       
Creates a unique ID for each host that forms a star. The format of                                                                                    
this ID is \<last snapshot where host was IDed\>_\<index at this snapshot\>.                                                                              
So, if a host was halo 5 at snapshot 3552 and then merged with halo 1                                                                                 
before the next snapshot, its unique ID will be 3552_5. Stars that form                                                                               
in its main progenitors will also be associated with this ID. These IDs                                                                               
are written out to a file with a similar format to <sim>_halostarhosts.txt.                                                                           
                                                                                                                                                      
Output: <sim>_uniquehalostarhosts.txt                                                                                                                 
                                                                                                                                                      
Usage:   python IDUniqueHost_rz.py <sim>                                                                                                              
Example: python IDUniqueHost_rz.py r634                                                                                                               
                                                                                                                                                      
Note that this is currently set up for MMs, but should be easily adapted                                                                              
by e.g., changing the paths or adding a path CL argument. It is also                                                                                  
designed to accommodate the phantoms that rockstar generates when it                                                                                  
temporarily loses track of a halo, which slows it down quite a bit.                                                                                   
If you're only ever going to be using it with other types of merger                                                                                   
trees, it can be simplified.  

Step 4 of stellar halo pipeline

**What it does:**
- Reads the `<sim>_halostarhosts.txt` file created in Step 3 (which lists halos that formed stars at each snapshot)
- For each star-forming halo, traces its merger history forward in time using Tangos database merger trees
- Creates a unique, persistent ID for each halo that accounts for mergers and halo evolution
- Assigns the same unique ID to stars formed in progenitor halos that later merge
- Handles "phantom" halos (temporary tracking losses in halo finders) for robust merger tree following
- Outputs a mapping file that connects local halo IDs to persistent unique IDs

**Detailed Process:**
1. **Input Parsing**: Reads the timeline of star-forming halos from `_halostarhosts.txt`
2. **Merger Tree Traversal**: For each halo, uses Tangos database to trace descendants forward in time
3. **Self-Consistency Checking**: Verifies that merger tree connections are bidirectional (descendant→progenitor matches)
4. **Unique ID Assignment**: Creates IDs in format `<last_snapshot>_<halo_index>` where the halo was last independently identified
5. **Progenitor Chain Building**: Links all progenitors in a merger chain to the same unique ID
6. **Phantom Handling**: Accommodates temporary halo finder failures using robust tree traversal algorithms

**Key Technical Details:**
- **Unique ID Format**: `SSSS_H` where `SSSS` = 4-digit snapshot number, `H` = halo index at that snapshot
- **Forward Tracking**: Uses `trackforward()` function to find the last snapshot where a halo exists independently
- **Merger Tree Validation**: Employs `checkmatch_p()` and `checkmatch_d()` to verify progenitor/descendant relationships
- **Phantom Accommodation**: Filters out phantom halos (type > 0) but maintains merger tree integrity
- **Caching System**: Uses dictionary `d` to store previously computed unique IDs for efficiency

**Example Unique ID Creation:**
```
Halo 5 at snapshot 3552 merges with halo 1 before snapshot 3553
→ Unique ID: "3552_5"
→ All stars formed in this halo's progenitors get ID "3552_5"
→ Stars formed in halo 1 (after merger) get a different unique ID
```

**Algorithm Workflow:**
1. **For each timestep** in the simulation:
   - **For each halo** that formed stars at that timestep:
     - Check if unique ID already computed (use cached result)
     - If not cached: trace forward to find last independent existence
     - Create unique ID based on final independent snapshot
     - Trace backward through progenitor chain
     - Assign same unique ID to all progenitors in the chain
     - Cache results for future lookups

**Merger Tree Functions:**
- **`trackforward(step, halo)`**: Traces halo forward to find last independent snapshot
- **`checkmatch_p(step, halo, hid, disp)`**: Verifies progenitor relationship
- **`checkmatch_d(step, halo, hid, disp)`**: Verifies descendant relationship

**Input Files:**
- `<sim>_halostarhosts.txt` from Step 3 (timeline of star-forming halos)
- Tangos database with merger tree information

**Output File:** `<sim>_uniquehalostarhosts.txt`
- Text file with same format as input but with unique IDs replacing local halo indices
- Format: `<timestep>\t<unique_id>,<local_halo_id>[,<star_count>]\t...`
- Example line: `3840    3552_5,15,234    3721_42,42,156`
  - At snapshot 3840: unique halo "3552_5" (local ID 15) formed 234 stars

<!-- **Performance Considerations:**
- **Caching**: Avoids recomputing unique IDs for halos already processed
- **Phantom Handling**: Designed for Rockstar halo finder but slows processing
- **Database Queries**: Intensive use of Tangos merger tree calculations
- **Memory Usage**: Stores merger tree data and caching dictionary -->

**Data Flow:**
```
Step 3 Output: *_halostarhosts.txt (local halo IDs)
                    ↓
Step 4: Merger tree analysis + unique ID assignment
                    ↓
Step 4 Output: *_uniquehalostarhosts.txt (persistent unique IDs)
```

**Special Cases Handled:**
- **Unbound Stars**: Particles with host ID = -1 get unique ID `<snapshot>_0`
- **Phantom Halos**: Temporary tracking losses in halo finder are filtered but accounted for
- **Merger Events**: Multiple local halos can map to the same unique ID if they're part of the same merger tree
- **Isolated Halos**: Halos that never merge retain their snapshot-specific unique ID

**Purpose:** This step solves the fundamental problem that halo IDs change over time due to mergers, making it impossible to track stellar populations. By creating persistent unique IDs, we can:
- Group stars by their true formation halo, even after mergers
- Trace stellar populations through cosmic time
- Identify which stars belong to the main galaxy vs. accreted satellites
- Enable stellar halo analysis based on formation environment

* Usage: `python IDUniqueHost_rz.py <sim> <tangos_simulation> <input_file> <output_file>`
* Example: `python IDUniqueHost_rz.py cptmarvel.cosmo25cmb.4096g5HbwK1BH sim_object halostarhosts.txt uniquehalostarhosts.txt`
* Runtime: ~2 minutes 

<!-- **Important Notes:**
- **Computationally Intensive**: Merger tree queries can be slow for large simulations
- **Rockstar Optimized**: Phantom handling is specific to Rockstar halo finder behavior
- **Database Dependent**: Requires properly constructed Tangos merger trees
- **Bidirectional Verification**: Ensures merger tree consistency through forward/backward checking
- **Memory Scaling**: Caching dictionary grows with number of unique halos processed -->

**Output Validation:**
The output file enables Step 5 to create a comprehensive database where every star particle is assigned to a persistent halo ID, regardless of when it formed or how many mergers occurred afterward.

<!-- ### 4) IDUniqueHost_rz.py

Step 4 of stellar halo pipeline

**What it does:**
- Reads the `<sim>_halostarhosts.txt` file created in Step 3 (timeline of star-forming halos)
- For each star-forming halo, traces its merger history forward in time using Tangos database merger trees
- Creates a unique, persistent ID for each halo that accounts for mergers and halo evolution  
- Assigns the same unique ID to stars formed in progenitor halos that later merge
- Handles "phantom" halos (temporary tracking losses in halo finders) for robust merger tree following

**Input:** `<sim>_halostarhosts.txt` from Step 3 (timeline of star-forming halos with local IDs)

**Output:** `<sim>_uniquehalostarhosts.txt` - Text file containing:
- Same format as input but with unique IDs replacing local halo indices
- Format: `<timestep>\t<unique_id>,<local_halo_id>,<star_count>\t...`
- Unique ID format: `SSSS_H` where `SSSS` = 4-digit snapshot number, `H` = halo index

**Purpose:** This solves the fundamental problem that halo IDs change over time due to mergers. By creating persistent unique IDs, we can group stars by their true formation halo even after mergers, enabling stellar halo analysis based on formation environment.

* Usage: `python IDUniqueHost_rz.py <sim> <tangos_simulation> <input_file> <output_file>`
* Example: `python IDUniqueHost_rz.py cptmarvel.cosmo25cmb.4096g5HbwK1BH sim_object halostarhosts.txt uniquehalostarhosts.txt` -->

In [36]:
import IDUniqueHost_rz
# import unique_host_ids_modified_Version2

In [37]:
sim = tangos.get_simulation(ss_dir)
print(f"Simulation: {sim}")
import collections
d = collections.defaultdict(list)
print(d)

hsfile = os.path.join(outfile_dir, f"{basename}_halostarhosts.txt")
ofile = os.path.join(outfile_dir, f"{basename}_uniquehalostarhosts.txt")

IDUniqueHost_rz.main(sim, d, hsfile, ofile)

Simulation: <Simulation("storm.4096g5HbwK1BH_bn")>
defaultdict(<class 'list'>, {})
------ 192
Current: 0192_1
Unique:  4096_7
Current: 0192_2
Unique:  4096_2
Current: 0192_3
Unique:  3744_13
Current: 0192_4
Unique:  4096_8
Current: 0192_5
Unique:  0576_14
Current: 0192_6
Unique:  1056_27
Current: 0192_7
Unique:  4096_3
Current: 0192_8
Unique:  0960_8
Current: 0192_9
Unique:  0480_43
Current: 0192_10
Unique:  1440_49
Current: 0192_11
Unique:  4096_16
Current: 0192_12
Unique:  0576_23
Current: 0192_13
Unique:  2400_28
Current: 0192_14
Unique:  4096_5
Current: 0192_15
Unique:  2304_133
Current: 0192_16
Unique:  0480_61
Current: 0192_18
Unique:  0480_15
Current: 0192_19
Unique:  4096_49
Current: 0192_20
Unique:  1632_6
Current: 0192_21
Unique:  1056_364
Current: 0192_22
Unique:  4096_43
Current: 0192_23
Unique:  3648_90
Current: 0192_24
Unique:  4096_218
Current: 0192_25
Unique:  0192_25
Current: 0192_26
Unique:  4096_23
Current: 0192_27
Unique:  0672_9
Current: 0192_29
Unique:  2400_29
Cu

In [ ]:
# unique_host_ids_modified_Version2.main(sim, d, hsfile, ofile)

### 5) StoreUniqueHostID_rz.py

Step 5 of stellar halo pipeline
Stores the unique ID of each star particle's host at formation time.
Creates an hdf5 file that contains this in addition to all of the data
from the <sim>_stardata_<snapshot>.h5 files. Note that all star particles
that don't have a host in the snapshot after they formed will be assigned 
a unique ID of <snapshot_index>_0 and a particle host (i.e., host at
formation time) of -1. It is recommended that you use the TrackDownStars
Jupyter notebook to try to manually identify hosts for these stars and then
use FixHostIDs_rz.py to amend <sim>_allhalostardata.h5.

Output: <sim>_allhalostardata.h5

Usage:   python StoreUniqueHostID_rz.py <sim>
Example: python StoreUniqueHostID_rz.py r634 

Note that this is currently set up for MMs, but should be easily adapted 
by e.g., changing the paths or adding a path CL argument.



Step 5 of stellar halo pipeline

**What it does:**
- Combines all `<sim>_stardata_*.h5` files from Step 2 into a single HDF5 file
- Maps each star particle's local host ID to its unique host ID from Step 4
- Creates final dataset linking every star particle to its persistent formation halo
- Handles unbound stars (host ID = -1) by assigning unique IDs like `3840_0`

**Input:** 
- Multiple `<sim>_stardata_*.h5` files from Step 2
- `<sim>_uniquehalostarhosts.txt` from Step 4 (halo ID mapping)

**Output:** `<sim>_allhalostardata.h5` containing:
- `particle_IDs`: Star particle IDs (`iord`)
- `particle_positions`: Formation positions (Mpc)
- `particle_creation_times`: Formation times (Gyr)
- `timestep_location`: Formation snapshot numbers
- `particle_hosts`: Local halo IDs at formation
- `host_IDs`: **Unique persistent halo IDs** (e.g., "3552_5")

**Key Process:**
1. Load unique ID mapping from Step 4: `"timestep,hostid" → "unique_id"`
2. For each star particle: lookup `(timestep, local_halo_id)` → assign unique ID
3. Unbound particles get default IDs: `f"{timestep:04d}_0"`
4. Combine all data into single compressed HDF5 file

**Purpose:** Creates analysis-ready dataset enabling stellar population studies with persistent halo tracking across mergers and cosmic time.

* Usage: `python StoreUniqueHostID_rz.py <sim> <output_directory>`
* Example: `python StoreUniqueHostID_rz.py cptmarvel.cosmo25cmb.4096g5HbwK1BH /output/path/`

In [38]:
import StoreUniqueHostID_rz

StoreUniqueHostID_rz.main(basename, outfile_dir)

/home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_stardata_003072.h5 <KeysViewHDF5 ['particle_IDs', 'particle_creation_times', 'particle_hosts', 'particle_positions', 'timestep_location']>
/home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_stardata_000288.h5 <KeysViewHDF5 ['particle_IDs', 'particle_creation_times', 'particle_hosts', 'particle_positions', 'timestep_location']>
/home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_stardata_001248.h5 <KeysViewHDF5 ['particle_IDs', 'particle_creation_times', 'particle_hosts', 'particle_positions', 'timestep_location']>
/home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_stardata_002496.h5 <KeysViewHDF5 ['particle_IDs', 'particle_creation_times', 'particle_hosts', 'particle_positions', 'timestep_location']>
/home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_stardata_002976.h5 <KeysViewHDF5 ['particle_IDs', 'particle_creation_times', 'p

### 6a) TrackDownStars_rz.ipnb

### 6b) FixHostID_rz

Optional: Step 6b of stellar halo pipeline
Updates the host_ID values stored in the allhalostardata hdf5 file based
on user input. This is designed as a follow-up to TrackDownStars and can 
be used in a couple of ways. If ffile=True, this script will look for 
numpy files with names <sim>_new_ID_?.npy and will assign the particles
with the iords in a given file to new_ID. If ffile=False, it will assign
all particles with a host_ID in the old_ID list to new_ID.

Output: <sim>_allhalostardata_upd.h5

Usage:   python FixHostIDs_rz.py <sim>
Example: python FixHostIDs_rz.py r634 

The script will print out all host_IDs for which the number of assigned particles 
changed and how many particles each gained/lost. If the output looks correct,
the user should manually rename <sim>_allhalostardata_upd.h5 to <sim>_allhalostardata.h5.
It's often necessary to go back and forth between this and TrackDownStars, in which 
case I usually move the *.npy files that have already been processed to a subfolder. 


In [7]:
import FixHostIDs_rz

In [8]:
FixHostIDs_rz.main(outfile_dir, basename)

0384_47: 757 particles lost
0672_112: 33 particles lost
0960_758: 379 particles lost
1056_27: 2218 particles gained
1152_26: 1681 particles lost
2304_14: 7290 particles gained
2496_127: 214 particles lost
2496_224: 109 particles lost
2496_61: 5581 particles lost
2592_193: 217 particles lost
4096_4: 537 particles lost
---------------------------------
If this looks correct, run
mv /home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_allhalostardata_upd.h5 /home/ns1917/pynbody/stellarhalo_trace_aw/storm.cosmo25cmb.4096g5HbwK1BH_allhalostardata.h5


In [20]:
# Read in your data
with h5py.File(outfile_dir+'/'+basename+'_allhalostardata_upd.h5','r') as f:
    hostids = f['host_IDs'].asstr()[:] # unique host IDs
    partids = f['particle_IDs'][:] # iords
    pct = f['particle_creation_times'][:] # formation times
    ph = f['particle_hosts'][:] # local host IDs (i.e., host at formation time)
    pp = f['particle_positions'][:] # position at formation time
    tsloc = f['timestep_location'][:] # snapshot where star particle first appears
uIDs = np.unique(hostids)

In [14]:
df = pd.DataFrame({
    'host_IDs': hostids,
    'particle_IDs': partids,
    # 'particle_creation_times': pct,
    # 'particle_hosts': ph,
    # 'particle_positions': pp,
    # 'timestep_location': tsloc
})
# to csv
df.to_csv(outfile_dir+'/final_'+basename+'_allhalostardata.csv', index=False)

### 6c) CompTwoHalos_rz

Optional: Step 6c of stellar halo pipeline
Compares two halos to see how likely it is that one is
the main progenitor of the other based on how many particles
they have in common. This is particularly useful when you've
used a merger tree constructor that doesn't use phantoms or
some equivalent and may therefore fail to connect a halo at
snapshot1 to the same halo at snapshot3 if it lost track of it
at snapshot2. This information can then be used with FixHostIDs_rz
to merge two unique IDs and/or to create a new link in the 
relevant tangos db.

Usage: python CompTwoHalos_rz.py <sim> <halo1> <halo2>
Example: python CompTwoHalos_rz.py r718 0136_4 0192_3

Output: prints out the fraction of <halo1>'s DM particles
that are in <halo2> and vice-versa.

It takes three arguments: the simulation you're working with
and the tangos IDs of the two halos you want to compare, which
are formatted as <snapshot>_<IDatsnapshot>. Note that this ID
is assumed to be the tangos ID, not necessarily the amiga.grp
ID.


## Extras

### Some cosmology and pynbody syntax

In [103]:
# boxsize, mass unit, vel unit, h

import pynbody

print(os.listdir('/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn/'))

s = pynbody.load('/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn/cptmarvel.cosmo25cmb.4096g5HbwK1BH.004096')
s.properties

['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000482.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001792.igasorder', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002048.z0.824.AHF_fpos', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000768', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002048.amiga.grp.pynbody-meta', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002162', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001664.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003456.FeMassFrac', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000291.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003968.igasorder', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002176.z0.741.AHF_profiles', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000512', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003245.amiga.grp', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001025.z1.999.AHF_halos', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003328.OxMassFrac', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001162.trace_back_merge.hdf5', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002816.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.0006

{'omegaM0': 0.24,
 'omegaL0': 0.76,
 'h': 0.7299490542599526,
 'boxsize': Unit("2.50e+04 kpc a"),
 'a': 1.0000000000142635,
 'time': Unit("1.40e+01 s kpc km**-1")}

In [11]:
!tangos serve

Starting server in PID 67784.
2025-07-25 15:54:10,352 INFO  [waitress:449][MainThread] Serving on http://[::1]:6543
2025-07-25 15:54:10,352 INFO  [waitress:449][MainThread] Serving on http://127.0.0.1:6543
2025-07-25 15:54:21,074 : Tree build complete; total time 1.28s
2025-07-25 15:54:21,074 INFO  [tangos.log:72][waitress-0] Tree build complete; total time 1.28s
2025-07-25 15:54:21,074 :   Progenitor query took 1.09s
2025-07-25 15:54:21,074 INFO  [tangos.log:73][waitress-0]   Progenitor query took 1.09s
2025-07-25 15:54:21,074 :   Property query took 0.01s
2025-07-25 15:54:21,074 INFO  [tangos.log:74][waitress-0]   Property query took 0.01s
2025-07-25 15:54:21,074 :   Tree post-processing took 0.18s
2025-07-25 15:54:21,074 INFO  [tangos.log:75][waitress-0]   Tree post-processing took 0.18s
2025-07-25 15:54:31,899 : Tree build complete; total time 0.31s
2025-07-25 15:54:31,899 INFO  [tangos.log:72][waitress-0] Tree build complete; total time 0.31s
2025-07-25 15:54:31,899 :   Progenitor

In [105]:
stars = s.s

In [114]:
stars['age'].in_units('Gyr')

SimArray([ 1.35174602e+01,  1.35124357e+01,  1.35082486e+01, ...,
          -6.11688358e-05, -6.11688358e-05, -6.11688358e-05],
         shape=(414599,), 'Gyr')

In [115]:
stars['tform'].in_units('Gyr')

SimArray([ 0.21365837,  0.21868286,  0.22286992, ..., 13.7311797 ,
          13.7311797 , 13.7311797 ], shape=(414599,), 'Gyr')

In [118]:
halos = s.halos()

In [119]:
halos['pos'].in_units('kpc')

AttributeError: 'SubhaloCatalogue' object has no attribute 'in_units'

In [117]:
stars['pos'].in_units('kpc')

SimArray([[  188.98196286,  -441.26994908,  1448.17242401],
          [  189.13126551,  -440.93732723,  1447.86080348],
          [  189.34544642,  -440.64465911,  1448.39063289],
          ...,
          [  566.63681754,  -234.70097222,  1668.89894756],
          [ -563.58575822,  -739.46160265,  1163.97785025],
          [-1974.30718693, -2313.09272352,   533.08499046]],
         shape=(414599, 3), 'kpc')

In [112]:
stars['vel'].in_units('km s**-1')

SimArray([[ 35.8012164 , -61.13322961,  50.41188651],
          [ 30.08138783, -47.59994133,  40.65547787],
          [ 31.14195046, -61.25638715,  41.8654215 ],
          ...,
          [ 19.43312753,   9.88569869,  44.19992337],
          [ -9.5371297 , -59.62350931,  23.09402733],
          [-89.22849431, -72.43745011,  -7.20320855]], shape=(414599, 3), 'km s**-1')

In [109]:
stars.derivable_keys()

['HII',
 'HeIII',
 'ne',
 'hetot',
 'hydrogen',
 'feh',
 'oxh',
 'ofe',
 'mgfe',
 'nefe',
 'sife',
 'c_s',
 'c_s_turb',
 'mjeans',
 'mjeans_turb',
 'ljeans',
 'ljeans_turb',
 'U_mag',
 'U_lum_den',
 'B_mag',
 'B_lum_den',
 'V_mag',
 'V_lum_den',
 'R_mag',
 'R_lum_den',
 'I_mag',
 'I_lum_den',
 'J_mag',
 'J_lum_den',
 'H_mag',
 'H_lum_den',
 'K_mag',
 'K_lum_den',
 'u_mag',
 'u_lum_den',
 'g_mag',
 'g_lum_den',
 'r_mag',
 'r_lum_den',
 'i_mag',
 'i_lum_den',
 'z_mag',
 'z_lum_den',
 'y_mag',
 'y_lum_den',
 'r',
 'rxy',
 'vr',
 'v2',
 'vt',
 'ke',
 'te',
 'j',
 'j2',
 'jz',
 'vrxy',
 'vcxy',
 'vphi',
 'vtheta',
 'v_mean',
 'v_disp',
 'v_curl',
 'vorticity',
 'v_div',
 'age',
 'theta',
 'alt',
 'az',
 'cs',
 'mu',
 'p',
 'u',
 'temp',
 'zeldovich_offset',
 'aform',
 'tform',
 'iord_argsort',
 'smooth',
 'rho',
 'mass_holder',
 'HI_frac',
 'HI_mass',
 'HI_N']

In [11]:
print(s['mass'].units)
print(s['vel'].units)

2.31e+15 Msol
6.30e+02 km a s**-1


In [14]:
result = 'cosmo25cmb.4096g5HbwK1BH_stardata_002688.h5'
result.split('.')[-2][-6:]

'002688'

### Make new amiga.grp files

In [1]:
!tangos serve

Starting server in PID 2627414.
2025-08-04 15:01:22,991 INFO  [waitress:449][MainThread] Serving on http://[::1]:6543
2025-08-04 15:01:22,991 INFO  [waitress:449][MainThread] Serving on http://127.0.0.1:6543
^C


In [48]:
steps = [step.extension for step in tangos.get_simulation(ss_dir).timesteps]

In [49]:
def create_thread_groups(items, n_threads):
    """
    Split items into groups for parallel processing
    
    Parameters:
    -----------
    items : list or array
        Items to be processed (in your case, the steps)
    n_threads : int
        Number of threads/processes to use
        
    Returns:
    --------
    groups : list of lists
        Each sublist contains items for one thread to process
    """
    import math
    items = list(items)  # Ensure it's a list
    n_items = len(items)
    
    if n_threads >= n_items:
        # If more threads than items, give each item its own thread
        return [[item] for item in items]
    
    # Calculate items per thread
    items_per_thread = math.ceil(n_items / n_threads)
    
    groups = []
    for i in range(0, n_items, items_per_thread):
        group = items[i:i + items_per_thread]
        groups.append(group)
    
    return groups

import pynbody
import tqdm.auto as tqdm
import time

from multiprocessing import Pool

def process_step(step):
    if type(step) is not str:
        pbar = tqdm.tqdm(total=len(step))
        for st in step:
            _process_step(st)
            pbar.update(1)
        pbar.close()
    else:
        _process_step(step)
    return 0    

def _process_step(step):
    path = sim_base + step
    # print(path)
    # time.sleep(1)
    print('Loading <{}>'.format(step))
    f = pynbody.load(path)
    print('  Loading halos for <{}>'.format(step))
    try:
        h = f.halos(halo_numbers='v1')
        print('    Writing amiga.grp for <{}>'.format(step))
        f['amiga.grp'] = h.get_group_array()
        f['amiga.grp'].write(overwrite=True)
        print('    Finished writing amiga.grp for <{}>'.format(step))
    except Exception as e:
        print('  ERROR loading halos for <{}>: {}'.format(step, e))

In [50]:
n_threads = 14

groups = create_thread_groups(steps, n_threads=n_threads)
for group in groups:
    print(group)

['storm.cosmo25cmb.4096g5HbwK1BH.000192', 'storm.cosmo25cmb.4096g5HbwK1BH.000288', 'storm.cosmo25cmb.4096g5HbwK1BH.000384']
['storm.cosmo25cmb.4096g5HbwK1BH.000480', 'storm.cosmo25cmb.4096g5HbwK1BH.000576', 'storm.cosmo25cmb.4096g5HbwK1BH.000672']
['storm.cosmo25cmb.4096g5HbwK1BH.000768', 'storm.cosmo25cmb.4096g5HbwK1BH.000864', 'storm.cosmo25cmb.4096g5HbwK1BH.000960']
['storm.cosmo25cmb.4096g5HbwK1BH.001056', 'storm.cosmo25cmb.4096g5HbwK1BH.001152', 'storm.cosmo25cmb.4096g5HbwK1BH.001248']
['storm.cosmo25cmb.4096g5HbwK1BH.001344', 'storm.cosmo25cmb.4096g5HbwK1BH.001440', 'storm.cosmo25cmb.4096g5HbwK1BH.001536']
['storm.cosmo25cmb.4096g5HbwK1BH.001632', 'storm.cosmo25cmb.4096g5HbwK1BH.001728', 'storm.cosmo25cmb.4096g5HbwK1BH.001824']
['storm.cosmo25cmb.4096g5HbwK1BH.001920', 'storm.cosmo25cmb.4096g5HbwK1BH.002016', 'storm.cosmo25cmb.4096g5HbwK1BH.002112']
['storm.cosmo25cmb.4096g5HbwK1BH.002208', 'storm.cosmo25cmb.4096g5HbwK1BH.002304', 'storm.cosmo25cmb.4096g5HbwK1BH.002400']
['storm.

In [51]:
with Pool(processes=n_threads) as pool:
    results = pool.map(process_step, groups)

Loading <storm.cosmo25cmb.4096g5HbwK1BH.000768>Loading <storm.cosmo25cmb.4096g5HbwK1BH.003360>Loading <storm.cosmo25cmb.4096g5HbwK1BH.001920>Loading <storm.cosmo25cmb.4096g5HbwK1BH.000192>Loading <storm.cosmo25cmb.4096g5HbwK1BH.003072>




Loading <storm.cosmo25cmb.4096g5HbwK1BH.000480>Loading <storm.cosmo25cmb.4096g5HbwK1BH.002208>Loading <storm.cosmo25cmb.4096g5HbwK1BH.001344>Loading <storm.cosmo25cmb.4096g5HbwK1BH.002784>Loading <storm.cosmo25cmb.4096g5HbwK1BH.001632>Loading <storm.cosmo25cmb.4096g5HbwK1BH.001056>Loading <storm.cosmo25cmb.4096g5HbwK1BH.003648>


Loading <storm.cosmo25cmb.4096g5HbwK1BH.002496>



Loading <storm.cosmo25cmb.4096g5HbwK1BH.003936>

Loading <storm.cosmo25cmb.4096g5HbwK1BH.003360>Loading <storm.cosmo25cmb.4096g5HbwK1BH.001920>Loading <storm.cosmo25cmb.4096g5HbwK1BH.000192>Loading <storm.cosmo25cmb.4096g5HbwK1BH.003072>




Loading <storm.cosmo25cmb.4096g5HbwK1BH.000480>Loading <storm.cosmo25cmb.4096g5HbwK1BH.002208>Loading <storm.cosmo25cmb.4096g5HbwK1BH.0

In [52]:
fname = sim_base + basename+ '.004096'
print(fname)
s = pb.load(fname)
unique_gp = np.unique(s.s['amiga.grp'])
print(unique_gp)

/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.004096
[    1     2     3     4     5     6     7     8    10    11    13    14
    16    17    20    22    23    24    28    34    35    39    43    47
    49    50    60    72   109   124   125   169   192   208   218   253
   292  1004  2423  3127  3738  3809  5428  5490  5779  9399 13323 14526
 15823 16720]


### Image

In [8]:
!pip install imageio imageio-ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 50.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [imageio]m1/2 [imageio]


In [7]:
import imageio.v2 as imageio
import numpy as np
import glob
import os
import re

def create_animation_with_padding(input_folder, output_file, fps=10):
    """
    Creates a GIF or MP4 from a folder of PNG images with varying sizes.

    It finds the largest image dimensions and pads smaller images with black
    borders to create uniformly-sized frames, preventing distortion.
    
    Args:
        input_folder (str): Path to the folder containing the PNG images.
        output_file (str): Path for the output animation file. The extension 
                           (.gif or .mp4) determines the format.
        fps (int): Frames per second for the output animation.
    """
    search_path = os.path.join(input_folder, '*.png')
    filenames = glob.glob(search_path)

    if not filenames:
        print(f"Error: No PNG files found in '{input_folder}'")
        return

    # Sort files naturally to handle numbers like 'plot_2.png' before 'plot_10.png'
    def natural_sort_key(s):
        return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]
    
    filenames.sort(key=natural_sort_key)

    # --- Step 1: Scan all images to find the maximum dimensions ---
    print("Scanning images to determine maximum dimensions...")
    max_height = 0
    max_width = 0
    
    # Also determine the color channel configuration (RGB vs RGBA) from the first image
    first_image = imageio.imread(filenames[0])
    print(f"First image shape: {first_image.shape}")
    channels = first_image.shape[2] if len(first_image.shape) == 3 else 1
    dtype = first_image.dtype

    for filename in filenames:
        img = imageio.imread(filename)
        if len(img.shape) == 3: # Ensure it's a color image
            h, w, _ = img.shape
            max_height = max(max_height, h)
            max_width = max(max_width, w)

    print(f"All frames will be resized to {max_width}x{max_height} pixels.")

    # --- Step 2: Create the animation with padded frames ---
    print(f"Creating animation at '{output_file}'...")
    with imageio.get_writer(output_file, fps=fps) as writer:
        for filename in tqdm.tqdm(filenames, desc="Processing frames"):
            # Read the original image
            img = imageio.imread(filename)
            
            # Create a new black canvas with the target dimensions
            padded_frame = np.zeros((max_height, max_width, channels), dtype=dtype)
            
            # Calculate offsets to center the image
            h, w, _ = img.shape
            y_offset = (max_height - h) // 2
            x_offset = (max_width - w) // 2
            
            # Paste the original image onto the canvas
            padded_frame[y_offset:y_offset+h, x_offset:x_offset+w, :] = img
            
            # Append the padded frame to the video
            writer.append_data(padded_frame)

    print("Animation created successfully!")

def create_animation(input_folder, output_file, fps=10):
    """
    Creates a GIF or MP4 from a folder of PNG images.

    The function assumes filenames can be sorted naturally (e.g., frame_1.png, frame_2.png, ... frame_10.png).

    Args:
        input_folder (str): Path to the folder containing the PNG images.
        output_file (str): Path for the output animation file. The extension (.gif or .mp4) determines the format.
        fps (int): Frames per second for the output animation.
    """
    search_path = os.path.join(input_folder, '*.png')
    filenames = glob.glob(search_path)

    if not filenames:
        print(f"Error: No PNG files found in the directory: {input_folder}")
        return

    filenames.sort(key=lambda x: str.split(str.split(x, '.')[-2], '_')[-1])  # Sort by the numeric part of the filename

    print(f"Found {len(filenames)} images. Creating animation at {output_file}...")

    # Create the animation
    with imageio.get_writer(output_file, fps=fps) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    print("Animation created successfully!")
    
# # --- Example Usage ---
# # Replace this with the actual path to your saved plots
# image_folder = '/home/selvani/MAP/pynbody/stellarhalo_trace_aw/merge_plots/3' 

# # Define the output file path (can be .gif or .mp4)
# output_path = os.path.join(image_folder, 'merger_animation.mp4')

# # Create the animation
# create_animation_with_padding(image_folder, output_path, fps=5)

In [8]:
image_folder = os.path.join(outfile_dir, 'merge_plots')

In [9]:
halo_num = 2

folders = [ss_dir+str(halo_num), ss_dir+str(halo_num)+'zoom', ss_dir+str(halo_num)+'morezoom']
for folder in folders:
# for folder in os.listdir(image_folder):
    folder_path = os.path.join(image_folder, folder)
    base_name = os.path.basename(folder_path)
    print(f'In folder {folder}')
    output_gif_path = os.path.join(image_folder, f'{base_name}_animation.gif')
    output_mp4_path = os.path.join(image_folder, f'{base_name}_animation.mp4')

    create_animation(folder_path, output_gif_path, fps=12)
    create_animation_with_padding(folder_path, output_mp4_path, fps=6)

In folder storm.4096g5HbwK1BH_bn2
Found 42 images. Creating animation at /home/ns1917/pynbody/stellarhalo_trace_aw/merge_plots/storm.4096g5HbwK1BH_bn2_animation.gif...
Animation created successfully!
Scanning images to determine maximum dimensions...
First image shape: (1841, 5390, 4)
All frames will be resized to 5390x1841 pixels.
Creating animation at '/home/ns1917/pynbody/stellarhalo_trace_aw/merge_plots/storm.4096g5HbwK1BH_bn2_animation.mp4'...


Processing frames:   0%|          | 0/42 [00:00<?, ?it/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (5390, 1841) to (5392, 1856) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation created successfully!
In folder storm.4096g5HbwK1BH_bn2zoom
Found 42 images. Creating animation at /home/ns1917/pynbody/stellarhalo_trace_aw/merge_plots/storm.4096g5HbwK1BH_bn2zoom_animation.gif...
Animation created successfully!
Scanning images to determine maximum dimensions...
First image shape: (1841, 5390, 4)
All frames will be resized to 5390x1841 pixels.
Creating animation at '/home/ns1917/pynbody/stellarhalo_trace_aw/merge_plots/storm.4096g5HbwK1BH_bn2zoom_animation.mp4'...


Processing frames:   0%|          | 0/42 [00:00<?, ?it/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (5390, 1841) to (5392, 1856) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation created successfully!
In folder storm.4096g5HbwK1BH_bn2morezoom
Found 42 images. Creating animation at /home/ns1917/pynbody/stellarhalo_trace_aw/merge_plots/storm.4096g5HbwK1BH_bn2morezoom_animation.gif...
Animation created successfully!
Scanning images to determine maximum dimensions...
First image shape: (1841, 5390, 4)
All frames will be resized to 5390x1841 pixels.
Creating animation at '/home/ns1917/pynbody/stellarhalo_trace_aw/merge_plots/storm.4096g5HbwK1BH_bn2morezoom_animation.mp4'...


Processing frames:   0%|          | 0/42 [00:00<?, ?it/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (5390, 1841) to (5392, 1856) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation created successfully!
